In [8]:
# Provide the path to your ChromeDriver executable
service = Service("chromedriver.exe")  # Replace with your chromedriver path
driver = webdriver.Chrome(service=service, options=options)

In [11]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [17]:
# Cozmetica web scraping
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Set up the Selenium WebDriver (using Chrome in this case)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Function to extract reviews and ratings from the product page
def extract_reviews_and_ratings(product_url):
    driver.get(product_url)
    time.sleep(3)  # Give time for the page to load

    reviews = []
    ratings = []
    
    while True:
        # Extract review text and ratings from the current page
        review_elements = driver.find_elements(By.CSS_SELECTOR, '.jdgm-rev__body')  # Assuming the class is 'review-text'
        rating_elements = driver.find_elements(By.CSS_SELECTOR, '.jdgm-rev__rating')  # Assuming the class is 'review-rating'

        for review, rating in zip(review_elements, rating_elements):
            reviews.append(review.text.strip())
            ratings.append(rating.text.strip())  # Adjust if ratings are in a different format
        
        # Check if there is a "next" page button
        next_page_button = driver.find_elements(By.CSS_SELECTOR, '.jdgm-paginate__next-page')
        
        if next_page_button:
            # If the "next" page button exists, click it to go to the next page
            next_page_button[0].click()
            time.sleep(3)  # Wait for the next set of reviews to load
        else:
            # If no next page button, exit the loop
            break
    
    return reviews, ratings

# Function to extract product price (sale or regular)
def extract_price(product):
    try:
        # Check if sale price is present
        sale_price_element = product.find_element(By.CSS_SELECTOR, '.price-item--sale')
        if sale_price_element:
            return sale_price_element.text.strip()
    except Exception:
        pass

    try:
        # Fallback to regular price
        regular_price_element = product.find_element(By.CSS_SELECTOR, '.price-item--regular')
        if regular_price_element:
            return regular_price_element.text.strip()
    except Exception:
        pass

    # Return fallback if no price found
    return "Price Not Available"

# List of brand URLs to scrape
urls = [
    "https://cozmetica.pk/search?q=conatural&options%5Bprefix%5D=last&type=product",    #Conatural
    "https://cozmetica.pk/search?q=cosrx&options%5Bprefix%5D=last&type=product",       #Cosrx
    "https://cozmetica.pk/search?q=the+ordinary&options%5Bprefix%5D=last&type=product" #The Ordinary
    "https://cozmetica.pk/search?q=inkey+list&options%5Bprefix%5D=last&type=product",  # The INKEY List
    "https://cozmetica.pk/search?q=neutrogena&options%5Bprefix%5D=last&type=product"  # Neutrogena
    "https://cozmetica.pk/search?type=cera+ve&options%5Bprefix%5D=last&q=ceraVe"       #ceraVe
    
]

# Open CSV file for writing
with open('cozmetica_product_reviews_and_ratings.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Brand', 'Name', 'Price', 'ReviewText', 'Rating'])  # Write header row

    # Iterate over each brand URL
    for url in urls:
        driver.get(url)
        time.sleep(3)  # Wait for the page to load

        # Find all product items in the list
        products = driver.find_elements(By.CSS_SELECTOR, 'li.product')

        # Scrape product name, price, and URL from the current page
        product_list = []

        for product in products:
            try:
                product_data = {
                    "Brand": url.split('=')[1].split('&')[0].replace('+', ' ').title(),  # Extract brand from the URL
                    "Name": product.find_element(By.CSS_SELECTOR, '.card-title span').text.strip(),
                    "Price": extract_price(product),  # Extract price using the function
                    "URL": product.find_element(By.CSS_SELECTOR, '.card-link').get_attribute('href')
                }
                product_list.append(product_data)
            except Exception as e:
                print(f"Error extracting product data: {e}")

        # Extract reviews and ratings for each product in the product list
        for product in product_list:
            product_name = product['Name']
            product_price = product['Price']
            product_url = product['URL']
            product_brand = product['Brand']
            
            # Extract reviews and ratings for the product
            reviews, ratings = extract_reviews_and_ratings(product_url)

            # Write data to CSV for each review and its corresponding rating
            for review, rating in zip(reviews, ratings):
                writer.writerow([product_brand, product_name, product_price, review, rating])
            print(f"Processed reviews and ratings for {product_name}")

# Close the WebDriver
driver.quit()


Processed reviews and ratings for Conatural Organic Sweet Almond Oil - 250 ML
Processed reviews and ratings for Conatural Rose Water - 60ml
Processed reviews and ratings for Conatural Hair Conditioner
Processed reviews and ratings for Conatural Organic Age Defying Cream
Processed reviews and ratings for Conatural Sweet Orange Essential Oil
Processed reviews and ratings for Conatural Organic Avocado Oil 120ML
Processed reviews and ratings for Conatural Argan Oil (100ml)
Processed reviews and ratings for Conatural Geranium Essential Oil
Processed reviews and ratings for CoNatural Bergamot Essential Oil
Processed reviews and ratings for Conatural Citronella Essential Oil
Processed reviews and ratings for Conatural Tea Tree Essential Oil
Processed reviews and ratings for Conatural Lemon Essential Oil
Processed reviews and ratings for Conatural Essential Day Cream
Processed reviews and ratings for Conatural Lemon Grass Essential Oil
Processed reviews and ratings for Conatural Saffron Comple

In [44]:
#judge.me review scraping
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the Selenium WebDriver (using Chrome in this case)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Function to extract reviews and ratings from the product page
def extract_reviews_and_ratings(product_url):
    driver.get(product_url)
    time.sleep(3)  # Give time for the page to load

    reviews = []
    ratings = []
    # Scroll down to load all reviews
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Close the cookie bar if present
    try:
        cookie_button = driver.find_element(By.CSS_SELECTOR, '.jm-cookie-bar .btn--brand')
        if cookie_button.is_displayed():
            print("Cookie bar found, clicking to close...")
            cookie_button.click()
            time.sleep(1)  # Allow time for the overlay to close
    except Exception:
        pass
    try:
        form_element = driver.find_element(By.CSS_SELECTOR, 'form.search-navbar-form')
        if form_element.is_displayed():
            # Close or interact with the form to remove the blockage
            form_element.submit()  # Example: submitting the form
            time.sleep(1)  # Allow time for the form submission to complete
    except Exception as e:
        print("No form or unable to close form:", e)


    while True:
        # Extract review text and ratings from the current page
        review_elements = driver.find_elements(By.CSS_SELECTOR, '.review__body')  # Assuming the class is 'review-text'
        rating_elements = driver.find_elements(By.CSS_SELECTOR, '.star-rating.review__rating')  # Assuming the class is 'review-rating'

        for review, rating in zip(review_elements, rating_elements):
            reviews.append(review.text.strip())
            # Extract the score from the aria-label attribute
            aria_label = rating.get_attribute('aria-label')
            if aria_label:
                try:
                    ratings.append(float(aria_label.split()[0]))  # Extract the numerical part of the score
                except ValueError:
                    ratings.append(None)
            else:
                ratings.append(None)

        # Check if there is a "next" page button
        next_page_button = driver.find_elements(By.XPATH, '//span[contains(text(), "chevron_right") and not(contains(@class, "pagination__page-number--disabled"))]')
        
        if next_page_button:
            # If the "next" page button exists, click it to go to the next page
            # Use JavaScript to click the button directly
            driver.execute_script("arguments[0].click();", next_page_button[0])
            time.sleep(3)  # Wait for the next set of reviews to load
        else:
            # If no next page button, exit the loop (we're on the last page)
            break

    return reviews, ratings

# Function to extract product price
def extract_price(product):
    try:
        # Locate the price element
        price_element = product.find_element(By.CSS_SELECTOR, '.product-search-card__price')
        if price_element:
            return price_element.text.strip()  # Extract and clean the price text
    except Exception:
        pass

    # Return fallback if no price found
    return "Price Not Available"

# List of brand URLs to scrape
urls = [
    "https://judge.me/search?q=ordinary&page=1&sort_by=best_selling&category_id=2302&country=PK&store_country=%2a&min_rating=0&min_transparency=80&min_authenticity=0&currency=PKR&reviews_count=10&category_search=false",  # The Ordianry
    "https://judge.me/search?q=cosrx&page=1&category_id=2302&country=PK&store_country=%2a&min_rating=0&min_transparency=80&min_authenticity=0&reviews_count=20&category_search=false", #Cosrx
    "https://judge.me/search?q=the%20body%20shop&page=1&sort_by=best_selling&category_id=2302&country=PK&store_country=%2a&min_rating=0&min_transparency=80&min_authenticity=0&reviews_count=1&category_search=false", #the body shop
    "https://judge.me/search?q=neutrogena&page=1&sort_by=best_selling&category_id=2302&country=PK&store_country=%2a&min_rating=0&min_transparency=80&min_authenticity=0&currency=PKR&reviews_count=10&category_search=false", #neutrogena
    "https://judge.me/search?q=cerave&page=1&category_id=2302&country=PK&store_country=%2a&min_rating=0&min_transparency=80&min_authenticity=0&currency=PKR&reviews_count=10&category_search=false", #CeraVe
    "https://judge.me/search?q=conatural&page=1&sort_by=best_selling&category_id=2302&country=PK&store_country=%2a&min_rating=0&min_transparency=80&min_authenticity=0&reviews_count=1&category_search=false" #Conatural
]

# Open CSV file for writing
with open('judgeme_product_reviews_and_ratings.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Brand', 'Name', 'Price', 'ReviewText', 'Rating'])  # Write header row

    # Iterate over each brand URL
    for url in urls:
        driver.get(url)
        time.sleep(3)  # Wait for the page to load

        # Close the cookie bar on the main page
        try:
            cookie_button = driver.find_element(By.CSS_SELECTOR, '.jm-cookie-bar .btn--brand')
            if cookie_button.is_displayed():
                print("Cookie bar found on main page, clicking to close...")
                cookie_button.click()
                time.sleep(1)  # Allow time for the overlay to close
        except Exception:
            print("No cookie bar found on main page or unable to click.")

        # Find all product items in the list
        products = driver.find_elements(By.CSS_SELECTOR, 'a.product-search-card')

        # Scrape product name, price, and URL from the current page
        product_list = []

        for product in products:
            try:
                product_data = {
                    "Brand": url.split('=')[1].split('&')[0].replace('+', ' ').title(),  # Extract brand from the URL
                    "Name": product.find_element(By.CSS_SELECTOR, '.product-search-card__title').text.strip(),    # Extract product name
                    "Price": extract_price(product),  # Extract price using the function
                    "URL": product.get_attribute('href')
                }
                product_list.append(product_data)
            except Exception as e:
                print(f"Error extracting product data: {e}")

        # Extract reviews and ratings for each product in the product list
        for product in product_list:
            product_name = product['Name']
            product_price = product['Price']
            product_url = product['URL']
            product_brand = product['Brand']
            
            # Extract reviews and ratings for the product
            reviews, ratings = extract_reviews_and_ratings(product_url)

            # Write data to CSV for each review and its corresponding rating
            for review, rating in zip(reviews, ratings):
                writer.writerow([product_brand, product_name, product_price, review, rating])
            print(f"Processed reviews and ratings for {product_name}")

# Close the WebDriver
driver.quit()


Cookie bar found on main page, clicking to close...
Processed reviews and ratings for The Ordinary- Hyaluronic Acid 2% + B5
Processed reviews and ratings for The Ordinary- Retinol 1% In Squalane 30Ml
Processed reviews and ratings for The Ordinary- Niacinamide 10% + Zinc 1% 30Ml
Processed reviews and ratings for The Ordinary Alpha Arbutin 2% + HA
Processed reviews and ratings for The Ordinary Lactic Acid 5% + HA
Processed reviews and ratings for THE ORDINARY Niacinamide 10% + Zinc 1%
Processed reviews and ratings for The Ordinary Salicylic Acid 2% Solution
Processed reviews and ratings for THE ORDINARY - NIACINAMIDE 10% + Zinc 1%
Processed reviews and ratings for The Ordinary Lactic Acid 10% + HA 2%
Processed reviews and ratings for TO Niacinamide 10% + Zinc 1% (60ml)
Processed reviews and ratings for TO Niacinamide Serum
Processed reviews and ratings for TO Alpha Arbutin 2% + HA
Processed reviews and ratings for TO AHA 30% + BHA 2% Peeling Solution
Processed reviews and ratings for TO 

Processed reviews and ratings for The Moms Co. Natural Baby Lotion with Rice Bran, Apricot, Jojoba Oil, Cocoa and Shea Butter


In [11]:
# UltaBeauty web scraping
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

# Set up the Selenium WebDriver (using Chrome in this case)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Function to close the cookie banner
def close_cookie_banner(driver):
    try:
        # Wait for the cookie banner to appear
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))
        )
        # Locate the "I Understand" button
        cookie_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
        # Click the button to accept cookies
        ActionChains(driver).move_to_element(cookie_button).click().perform()
        print("Cookie banner closed.")
    except Exception as e:
        print(f"Cookie banner not found or couldn't be closed: {e}")

def scroll_to_reviews_section(driver):
    """
    Scroll to the reviews section and ensure it is loaded.
    """
    for _ in range(20):  # Try up to 20 small scrolls
        try:
            review_section = driver.find_element(By.CSS_SELECTOR, '.pr-rd-description-text')
            if review_section.is_displayed():
                print("Reviews section loaded.")
                return True
        except:
            pass
        
        # Scroll down slightly
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)  # Allow time for content to load
    
    print("Failed to load reviews section.")
    return False


def extract_reviews_and_ratings(product_url, max_reviews=1000):
    driver.get(product_url)
    
    # Ensure the reviews section is visible
    if not scroll_to_reviews_section(driver):
        print(f"Could not load reviews section for {product_url}")
        return [], []
    
    reviews = []
    ratings = []
    page_number = 1  # Track pages for debugging
    
    while len(reviews) < max_reviews:
        print(f"Processing page {page_number}...")
        
        try:
            # Extract reviews and ratings on the current page
            review_elements = driver.find_elements(By.CSS_SELECTOR, '.pr-rd-description-text')
            rating_elements = driver.find_elements(By.CSS_SELECTOR, '.pr-snippet-stars')
            
            for review, rating in zip(review_elements, rating_elements):
                if len(reviews) >= max_reviews:
                    break
                reviews.append(review.text.strip())
                rating_text = rating.get_attribute("aria-label")
                if rating_text:
                    ratings.append(rating_text.split(' ')[1])  # Extract rating value
                else:
                    ratings.append(None)
            
            print(f"Collected {len(reviews)} reviews so far...")
            
            # Locate and click the next page button
            next_page_button = driver.find_elements(By.CSS_SELECTOR, '.pr-rd-pagination-btn--next')
            
            if next_page_button and next_page_button[0].is_enabled():
                # Ensure the button is visible before clicking
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button[0])
                time.sleep(3)  # Allow time for the button to stabilize
                next_page_button[0].click()
                
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.pr-rd-description-text'))
                )
                page_number += 1
            else:
                print("No more pages available.")
                break
        
        except (StaleElementReferenceException, TimeoutException) as e:
            print(f"Encountered an issue: {e}. Stopping...")
            break
    
    print(f"Finished collecting {len(reviews)} reviews.")
    return reviews[:max_reviews], ratings[:max_reviews]


# List of brand URLs to scrape
urls = [
    #"https://www.ulta.com/brand/body-shop",
    #"https://www.ulta.com/brand/neutrogena",
    #"https://www.ulta.com/brand/cosrx",
    #"https://www.ulta.com/brand/neutrogena",
    #"https://www.ulta.com/brand/ordinary",
    "https://www.ulta.com/brand/cerave",
    
]

# Function to extract product price
def extract_price(product):
    try:
        # Locate the price element
        price_element = product.find_element(By.CSS_SELECTOR, '.ProductPricing .Text-ds--body-2')
        if price_element:
            return price_element.text.strip()  # Extract and clean the price text
    except Exception:
        pass
    return "Price Not Available"

# Open CSV file for writing
with open('UltaBeauty_trial_product_reviews_and_ratings.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Brand', 'Name', 'Price', 'ReviewText', 'Rating'])  # Write header row

    # Iterate over each brand URL
    for url in urls:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'li.ProductListingResults__productCard'))
        )  # Wait for product listings to load
        # Close the cookie banner
        close_cookie_banner(driver)

        # Find all product items in the list
        products = driver.find_elements(By.CSS_SELECTOR, 'li.ProductListingResults__productCard')

        product_list = []
        for product in products:
            try:
                product_data = {
                    'Brand': product.find_element(By.CSS_SELECTOR, ".ProductCard__brand .Text-ds--body-2").text.strip(),
                    'Name': product.find_element(By.CSS_SELECTOR, ".ProductCard__product .Text-ds--body-2").text.strip(),
                    "Price": extract_price(product),  # Extract price using the function
                    'URL': product.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                }
                product_list.append(product_data)
            except Exception as e:
                print(f"Error extracting product data: {e}")

        # Extract reviews and ratings for each product in the product list
        for product in product_list:
            product_name = product['Name']
            product_price = product['Price']
            product_url = product['URL']
            product_brand = product['Brand']
            
            # Extract reviews and ratings for the product
            reviews, ratings = extract_reviews_and_ratings(product_url)

            # Write data to CSV for each review and its corresponding rating
            for review, rating in zip(reviews, ratings):
                writer.writerow([product_brand, product_name, product_price, review, rating])
            print(f"Processed reviews and ratings for {product_name}")

# Close the WebDriver
driver.quit()


Cookie banner closed.
Reviews section loaded.
Processing page 1...
Collected 5 reviews so far...
Processing page 2...
Collected 10 reviews so far...
Processing page 3...
Collected 15 reviews so far...
Processing page 4...
Collected 20 reviews so far...
Processing page 5...
Collected 25 reviews so far...
Processing page 6...
Collected 30 reviews so far...
Processing page 7...
Collected 35 reviews so far...
Processing page 8...
Encountered an issue: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0032FD53+23747]
	(No symbol) [0x002B7D54]
	(No symbol) [0x0018BE53]
	(No symbol) [0x0019196B]
	(No symbol) [0x0019387A]
	(No symbol) [0x001938F7]
	(No symbol) [0x001CAE66]
	(No symbol) [0x001F1DFC]
	(No symbol) [0x001C59A4]
	(No symbol) [0x0

Reviews section loaded.
Processing page 1...
Collected 5 reviews so far...
Processing page 2...
Encountered an issue: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0032FD53+23747]
	(No symbol) [0x002B7D54]
	(No symbol) [0x0018BE53]
	(No symbol) [0x0019196B]
	(No symbol) [0x0019387A]
	(No symbol) [0x001938F7]
	(No symbol) [0x001CAE66]
	(No symbol) [0x001F1DFC]
	(No symbol) [0x001C59A4]
	(No symbol) [0x001F2094]
	(No symbol) [0x0020B41E]
	(No symbol) [0x001F1B96]
	(No symbol) [0x001C3F3C]
	(No symbol) [0x001C4EBD]
	GetHandleVerifier [0x0060AC73+3017699]
	GetHandleVerifier [0x0061B93B+3086507]
	GetHandleVerifier [0x006140F2+3055714]
	GetHandleVerifier [0x003C5AF0+637536]
	(No symbol) [0x002C0A5D]
	(No symbol) [0x002BDA28]
	(No symbo

ElementClickInterceptedException: Message: element click intercepted: Element <a href="https://www.ulta.com/p/skin-renewing-vitamin-c-serum-skin-brightening-antioxidant-face-serum-pimprod2006754?sku=2548800&amp;pr_rd_page=2" class="pr-rd-pagination-btn pr-rd-pagination-btn--next">...</a> is not clickable at point (870, 7). Other element would receive the click: <div class="GridContainer__content">...</div>
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x0032FD53+23747]
	(No symbol) [0x002B7D54]
	(No symbol) [0x0018BE53]
	(No symbol) [0x001D5C57]
	(No symbol) [0x001D40C3]
	(No symbol) [0x001D1CCD]
	(No symbol) [0x001D1034]
	(No symbol) [0x001C5F57]
	(No symbol) [0x001F1DFC]
	(No symbol) [0x001C59A4]
	(No symbol) [0x001F2094]
	(No symbol) [0x0020B41E]
	(No symbol) [0x001F1B96]
	(No symbol) [0x001C3F3C]
	(No symbol) [0x001C4EBD]
	GetHandleVerifier [0x0060AC73+3017699]
	GetHandleVerifier [0x0061B93B+3086507]
	GetHandleVerifier [0x006140F2+3055714]
	GetHandleVerifi

In [2]:
# Data Cleaning
import pandas as pd

In [3]:
cozmetica_df = pd.read_csv("cozmetica_product_reviews_and_ratings.csv")
judgeme_df = pd.read_csv("judgeme_product_reviews_and_ratings.csv")
ultabeauty_df1 = pd.read_csv("UltaBeauty_ordinary_product_reviews_and_ratings.csv")
ultabeauty_df2 = pd.read_csv("UltaBeauty_cerave_product_reviews_and_ratings.csv")
ultabeauty_df3 = pd.read_csv("UltaBeauty_cosrx_product_reviews_and_ratings.csv")
ultabeauty_df4 = pd.read_csv("UltaBeauty_bodyshop_product_reviews_and_ratings.csv")
ultabeauty_df5 = pd.read_csv("UltaBeauty_neutrogena_product_reviews_and_ratings.csv")

In [4]:
print("\nExploring Cozmetica dataframe")
print(cozmetica_df.head())
print(cozmetica_df.info())
print(cozmetica_df.describe())
print(cozmetica_df.shape)




Exploring Cozmetica dataframe
          Brand                                          Name     Price  \
0     Conatural                    Conatural Hair Conditioner       NaN   
1     Conatural              Conatural Tea Tree Essential Oil       NaN   
2     Conatural          Conatural Saffron Complexion Builder       NaN   
3         Cosrx   Cosrx Acne Pimple Master Patch (24 Patches)    Rs.849   
4  The Ordinary  The Ordinary Niacinamide 10% + Zinc 1% Serum  Rs.2,799   

                                          ReviewText  Rating  
0                         Conatural Hair Conditioner     NaN  
1                               Product is very nice     NaN  
2  Very good cream, not irritating, although has ...     NaN  
3        Cosrx Acne Pimple Master Patch (24 Patches)     NaN  
4       The Ordinary Niacinamide 10% + Zinc 1% Serum     NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dty

In [5]:
print("\nExploring Judgeme dataframe")
print(judgeme_df.head())
print(judgeme_df.info())
print(judgeme_df.describe())
print(judgeme_df.shape)


Exploring Judgeme dataframe
      Brand                                   Name         Price  \
0  Ordinary  The Ordinary- Hyaluronic Acid 2% + B5  PKR 3,999.00   
1  Ordinary  The Ordinary- Hyaluronic Acid 2% + B5  PKR 3,999.00   
2  Ordinary  The Ordinary- Hyaluronic Acid 2% + B5  PKR 3,999.00   
3  Ordinary  The Ordinary- Hyaluronic Acid 2% + B5  PKR 3,999.00   
4  Ordinary  The Ordinary- Hyaluronic Acid 2% + B5  PKR 3,999.00   

                                          ReviewText  Rating  
0  It's original and after just use of one week l...     5.0  
1  I have bought original product multiple times ...     1.0  
2  Bohat acha hai best for skin care bigners Rean...     5.0  
3  Excellent result, little amount is enough, goo...     5.0  
4  It really work and orignal product,pleased to ...     5.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3031 entries, 0 to 3030
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  ----

In [6]:
#merging ultabeauty dataframe
ultabeauty_df = pd.concat([ultabeauty_df1, ultabeauty_df2, ultabeauty_df3, ultabeauty_df4, ultabeauty_df5],ignore_index=True)
print("\nExploring UltaBeauty dataframe")
print(ultabeauty_df.head())
print(ultabeauty_df.info())
print(ultabeauty_df.describe())
print(ultabeauty_df.shape)


Exploring UltaBeauty dataframe
          Brand                                Name   Price  \
0  The Ordinary  Glycolic Acid 7% Exfoliating Toner  $13.00   
1  The Ordinary  Glycolic Acid 7% Exfoliating Toner  $13.00   
2  The Ordinary  Glycolic Acid 7% Exfoliating Toner  $13.00   
3  The Ordinary  Glycolic Acid 7% Exfoliating Toner  $13.00   
4  The Ordinary  Glycolic Acid 7% Exfoliating Toner  $13.00   

                                          ReviewText  Rating  
0  The reviews speak for themselves but this prod...     4.3  
1  I bought this the first time at the urging of ...     5.0  
2  I have been using this product for a year im o...     2.0  
3  I use this on face, neck and chest. Skin has a...     5.0  
4  I absolutely love this toner. It's clean, its ...     5.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9423 entries, 0 to 9422
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Brand       9423 n

In [7]:
df = pd.concat([cozmetica_df, judgeme_df, ultabeauty_df], ignore_index=True, verify_integrity = True)
print("\n Exploring complete dataset")
print(df.head())
print(df.info())
print(df.describe())
print(df.shape)


 Exploring complete dataset
          Brand                                          Name     Price  \
0     Conatural                    Conatural Hair Conditioner       NaN   
1     Conatural              Conatural Tea Tree Essential Oil       NaN   
2     Conatural          Conatural Saffron Complexion Builder       NaN   
3         Cosrx   Cosrx Acne Pimple Master Patch (24 Patches)    Rs.849   
4  The Ordinary  The Ordinary Niacinamide 10% + Zinc 1% Serum  Rs.2,799   

                                          ReviewText  Rating  
0                         Conatural Hair Conditioner     NaN  
1                               Product is very nice     NaN  
2  Very good cream, not irritating, although has ...     NaN  
3        Cosrx Acne Pimple Master Patch (24 Patches)     NaN  
4       The Ordinary Niacinamide 10% + Zinc 1% Serum     NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12712 entries, 0 to 12711
Data columns (total 5 columns):
 #   Column      Non-Null Count  D

In [8]:
Conatural_df = df[df['Brand'] == "Conatural"]
Conatural_products = Conatural_df['Name'].unique()
#print(Conatural_products)
mapping = {'Conatural Hair Conditioner':'Conatural Hair Conditioner',
           'Conatural Tea Tree Essential Oil':'Conatural Tea Tree Essential Oil',
            'Conatural Saffron Complexion Builder':'Conatural Saffron Complexion Builder',
          'Conatural Anti-Aging - Super Activs Skin Serum':'Conatural Anti-Aging - Super Activs Skin Serum',
          'CoNatural Super Activs Anti-Aging Skin Serum 30 ML':'Conatural Anti-Aging - Super Activs Skin Serum',
          'Conatural Anti-Pigmentation - Super Activs Skin Serum':'Conatural Anti-Pigmentation - Super Activs Skin Serum',
          'Conatural Hyaluronic Acid 2% + B5 - Super Activs Skin Serum':'Conatural Hyaluronic Acid 2% + B5 - Super Activs Skin Serum',
          'Conatural Niacinamide 10% + Zinc 1% - Super Activs Skin Serum':'Conatural Niacinamide 10% + Zinc 1% - Super Activs Skin Serum',
          'Co NATURAL Rose Face Wash':'Conatural Rose Face Wash'}
Conatural_df['Name'] = Conatural_df['Name'].replace(mapping)
print(Conatural_df['Name'].unique())

['Conatural Hair Conditioner' 'Conatural Tea Tree Essential Oil'
 'Conatural Saffron Complexion Builder'
 'Conatural Anti-Aging - Super Activs Skin Serum'
 'Conatural Anti-Pigmentation - Super Activs Skin Serum'
 'Conatural Hyaluronic Acid 2% + B5 - Super Activs Skin Serum'
 'Conatural Niacinamide 10% + Zinc 1% - Super Activs Skin Serum'
 'Conatural Rose Face Wash'
 'The Moms Co. Natural Vita Rich Face Serum with Vitamins and Hyaluronic Acid'
 'The Moms Co Natural Vita Rich Face Serum ( 8 ml ) ( Mini / Small ...'
 'The Moms Co. Natural Baby Lotion with Rice Bran, Apricot, Jojoba Oil, Cocoa and Shea Butter']


C:\Users\Hobopk\AppData\Local\Temp/ipykernel_16204/1892284048.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Conatural_df['Name'] = Conatural_df['Name'].replace(mapping)


In [9]:
df.update(Conatural_df)

In [10]:
unique_brands = df['Brand'].unique()
print(unique_brands)
brand_mapping = {'Conatural':'Conatural', 'Cosrx':'Cosrx', 'COSRX':'Cosrx', 'The Ordinary':'The Ordinary', 'Ordinary':'The Ordinary',
                'The%20Body%20Shop':'The Body Shop', 'The Body Shop':'The Body Shop', 'Cerave':'CeraVe', 'CeraVe':'CeraVe',
                 'Neutrogena':'Neutrogena'}
df['Brand'] = df['Brand'].replace(brand_mapping)
print(df['Brand'].unique())
df.update(df)

['Conatural' 'Cosrx' 'The Ordinary' 'Neutrogena' 'Ordinary'
 'The%20Body%20Shop' 'Cerave' 'CeraVe' 'COSRX' 'The Body Shop']
['Conatural' 'Cosrx' 'The Ordinary' 'Neutrogena' 'The Body Shop' 'CeraVe']


In [11]:
brands = df['Brand'].unique()
print(brands)
df.shape

['Conatural' 'Cosrx' 'The Ordinary' 'Neutrogena' 'The Body Shop' 'CeraVe']


(12712, 5)

In [12]:
Cosrx_df = df[df['Brand'] == "Cosrx"]
Cosrx_products = Cosrx_df['Name'].unique()
#print(Cosrx_products)
cosrx_mapping = {
    'Cosrx Acne Pimple Master Patch (24 Patches)':'Cosrx Acne Pimple Master Patch', 'Cosrx Acne Pimple Patch 24 Patches':'Cosrx Acne Pimple Master Patch',
    'COSRX Acne Pimple Master Patch':'Cosrx Acne Pimple Master Patch','COSRX Acne Pimple Master Patch (24pcs)':'Cosrx Acne Pimple Master Patch',
    'CosRX Acne Pimple Master Patch 24 Patches':'Cosrx Acne Pimple Master Patch', 'COSRX Acne Pimple Master Patch 24 patches':'Cosrx Acne Pimple Master Patch',
    'COSRX AHA BHA Clarifying Treatment Toner 150ml':'COSRX AHA/BHA Clarifying Treatment Toner',
    'CosRX Clear Fit Master Patch 18 Patches':'Cosrx Clear Fit Master Patch',
    'COSRX Snail 96 Mucin Essence 100ml':'Cosrx Advanced Snail 96 Mucin Power Essence',
    'Advanced Snail 96 Mucin Power Essence':'Cosrx Advanced Snail 96 Mucin Power Essence',
    'COSRX Advanced Snail 96 Mucin Power Essence 100ml':'Cosrx Advanced Snail 96 Mucin Power Essence',
    'COSRX Salicylic Acid Daily Gentle Cleanser': 'Cosrx Salicylic Acid Daily Gentle Cleanser',
    '[COSRX] Salicylic Acid Daily Gentle Cleanser 150ml':'Cosrx Salicylic Acid Daily Gentle Cleanser',
    'COSRX Low pH Good Morning Gel Cleanser':'Cosrx Low pH Good Morning Gel Cleanser',
    'COSRX Low-pH Good Morning Gel Cleanser (150ml)':'Cosrx Low pH Good Morning Gel Cleanser',
    'Cosrx Low PH Good Morning Gel Cleanser':'Cosrx Low pH Good Morning Gel Cleanser',
    '[COSRX] The Vitamin C 23 serum 20ml':'COSRX The Vitamin C 23 serum 20ml'
}
Cosrx_df['Name'] = Cosrx_df['Name'].replace(cosrx_mapping)
print(Cosrx_df['Name'].unique())
df.update(Cosrx_df)

['Cosrx Acne Pimple Master Patch'
 'COSRX AHA/BHA Clarifying Treatment Toner'
 'Cosrx One Step Original Clear Pad' 'Cosrx Clear Fit Master Patch'
 'Cosrx BHA Blackhead Power Liquid' 'COSRX The Vitamin C 23 serum 20ml'
 'Cosrx Advanced Snail 96 Mucin Power Essence'
 'Cosrx Salicylic Acid Daily Gentle Cleanser'
 'COSRX - OIL-FREE ULTRA-MOISTURIZING LOTION 100ml'
 'Cosrx Low pH Good Morning Gel Cleanser'
 'COSRX - Aloe Soothing Sun Cream SPF50+/ PA+++ 50ml'
 'Advanced Snail 92 All In One Cream']


C:\Users\Hobopk\AppData\Local\Temp/ipykernel_16204/2502484396.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cosrx_df['Name'] = Cosrx_df['Name'].replace(cosrx_mapping)


In [13]:
Cerave_df = df[df['Brand'] == "CeraVe"]
Cerave_products = Cerave_df['Name'].unique()
#print(Cerave_products)
cerave_mapping = {
    # Retinol Serums
    'CeraVe Resurfacing Retinol Serum': 'CeraVe Resurfacing Retinol Serum',
    '(PREORDER) CeraVe Resurfacing Retinol Serum': 'CeraVe Resurfacing Retinol Serum',
    'CeraVe Resurfacing Retinol Serum USA': 'CeraVe Resurfacing Retinol Serum',
    'Resurfacing Retinol Facial Serum for Acne Prone Skin': 'CeraVe Resurfacing Retinol Serum',

    # Moisturizers
    'CeraVe Daily Moisturizing Lotion': 'CeraVe Daily Moisturizing Lotion',
    'Daily Moisturizing Face and Body Lotion for Balanced to Dry Skin': 'CeraVe Daily Moisturizing Lotion',
    'Cerave Moisturizing Lotion Normal To Dry Skin USA': 'CeraVe Daily Moisturizing Lotion',
    'Moisturizing Cream, Body & Face Moisturizer for Balanced to Dry Skin': 'CeraVe Moisturizing Cream',
    'Moisturizing Cream with Pump for Balanced to Dry Skin': 'CeraVe Moisturizing Cream',
    'Travel Size Moisturizing Cream for Balanced to Dry Skin': 'CeraVe Moisturizing Cream',
    'Intensive Moisturizing Body Lotion for Dry Skin and Itch Relief': 'CeraVe Moisturizing Cream',
    'Ultra-Light Moisturizing Lotion with SPF 30 for Balanced to Oily Skin': 'CeraVe Ultra-Light Moisturizing Lotion',
    'AM Lotion Face Moisturizer with SPF 30 for Balanced to Oily Skin': 'CeraVe AM Lotion SPF 30',
    'AM Lotion Face Moisturizer with SPF 50 for Balanced to Oily Skin': 'CeraVe AM Lotion SPF 50',
    'PM Lotion Face Moisturizer, Oil-free Night Cream for All Skin Types': 'CeraVe PM Lotion',
    'Oil Control Moisturizing Gel-Cream Face Moisturizer for Oily Skin': 'CeraVe Oil Control Gel-Cream',
    
    # Facial Cleansers
    'CeraVe Hydrating Facial Cleanser': 'CeraVe Hydrating Facial Cleanser',
    'CeraVe Hydrating Cleanser - 236ml - New Release': 'CeraVe Hydrating Facial Cleanser',
    'CeraVe Hydrating Cleanser - 473ml - New Release': 'CeraVe Hydrating Facial Cleanser',
    'CeraVe Hydrating Cream To Foam Cleanser': 'CeraVe Hydrating Cream-to-Foam Cleanser',
    'CeraVe Hydrating Facial Cleanser 236 ml': 'CeraVe Hydrating Facial Cleanser',
    'Hydrating Facial Cleanser, Gentle Face Wash for Balanced to Dry Skin': 'CeraVe Hydrating Facial Cleanser',
    'Hydrating Cream-to-Foam Face Wash for Balanced to Dry Skin': 'CeraVe Hydrating Cream-to-Foam Cleanser',
    'Hydrating Foaming Cleansing Oil | سيرافي غسول زيتي': 'CeraVe Hydrating Foaming Oil Cleanser',
    'Foaming Facial Cleanser, Gel-Based Face Wash for Balanced to Oily Skin': 'CeraVe Foaming Facial Cleanser',
    'CeraVe Foaming Cleanser for Normal to oily Skin': 'CeraVe Foaming Facial Cleanser',
    'Travel Size Foaming Facial Cleanser for Balanced to Oily Skin': 'CeraVe Foaming Facial Cleanser',
    'Travel Size Hydrating Facial Cleanser for Balanced to Dry Skin': 'CeraVe Hydrating Facial Cleanser',
    'Hydrating Foaming Oil Cleanser, Water-Based Face Wash for Dry Skin': 'CeraVe Hydrating Foaming Oil Cleanser',
    'Acne Foaming Cream Cleanser with 4% BPO for Acne Prone Skin': 'CeraVe Acne Foaming Cream Cleanser',
    'Maximum Strength Acne Foaming Cream Wash with 10% BPO for Face & Body': 'CeraVe Acne Foaming Cream Wash',

    # SA Products
    'CERAVE RENEWING SA CLEANSER (US) 237ml': 'CeraVe Renewing SA Cleanser',
    'CeraVe Renewing SA Cleanser For Normal Skin USA': 'CeraVe Renewing SA Cleanser',
    'Renewing SA Cleanser, Salicylic Acid Face Wash for Balanced Skin': 'CeraVe Renewing SA Cleanser',
    'SA Cream, Salicylic Acid Body Cream for Rough & Bumpy Skin': 'CeraVe SA Cream',
    'SA Lotion, Salicylic Acid Body Lotion for Rough & Bumpy Skin': 'CeraVe SA Lotion',

    # Sunscreens
    'Hydrating Mineral Sunscreen Face Lotion with Sheer Tint SPF 30': 'CeraVe Mineral Sunscreen SPF 30',
    'Hydrating Mineral Sunscreen Lotion for Face SPF 50 for All Skin Types': 'CeraVe Mineral Sunscreen SPF 50',
    'Hydrating Sheer Sunscreen Face and Body Lotion with SPF 30': 'CeraVe Sheer Sunscreen SPF 30',
    'Hydrating Mineral Sunscreen Lotion for Body SPF 30 for All Skin Types': 'CeraVe Mineral Sunscreen SPF 30',
    'Hydrating Mineral Sunscreen Lotion for Body SPF 50 for All Skin Types': 'CeraVe Mineral Sunscreen SPF 50',

    # Miscellaneous
    'Skin Renewing Night Cream, Face Moisturizer for All Skin Types': 'CeraVe Skin Renewing Night Cream',
    'Skin Renewing Vitamin C Serum, Skin-Brightening Antioxidant Face Serum': 'CeraVe Skin Renewing Vitamin C Serum',
    'Skin Renewing Eye Cream, Anti-Aging Eye Treatment for All Skin Types': 'CeraVe Skin Renewing Eye Cream',
    'Skin Renewing Retinol Serum, Anti-Aging Face Serum for All Skin Types': 'CeraVe Skin Renewing Retinol Serum',
    'Skin Renewing Gel Oil, Anti-Aging Face Serum to Improve Skin Radiance': 'CeraVe Skin Renewing Gel Oil',
    'Skin Renewing Day Cream, Face Moisturizer with SPF 30 & Retinol': 'CeraVe Skin Renewing Day Cream',
    'Skin Renewing Nightly Exfoliating Treatment, Anti-Aging Face Serum': 'CeraVe Skin Renewing Nightly Exfoliating Treatment',
    'Hydrating Micellar Water with Ceramides for Dry Skin': 'CeraVe Hydrating Micellar Water',
    'Healing Ointment to Help Protect & Soothe Dry to Very Dry Skin': 'CeraVe Healing Ointment',
    'Ultra-Light Facial Gel with Hyaluronic Acid for All Skin Types': 'CeraVe Ultra-Light Gel',
    'Hydrating Hyaluronic Acid Face Serum with Vitamin B5 for Dry Skin': 'CeraVe Hydrating Hyaluronic Acid Serum',
    'Comforting Eye Makeup Remover with Hyaluronic Acid': 'CeraVe Comforting Eye Makeup Remover',
    'Acne Control Cleanser for Face with 2% Salicylic Acid & Purifying Clay for Oily Skin': 'CeraVe Acne Control Cleanser',
    'Acne Control Gel Blemish Treatment with AHA & BHA for Acne Prone Skin': 'CeraVe Acne Control Gel',
    'Plant-Based Hydrating Makeup Removing Face Wipes for All Skin Types': 'CeraVe Makeup Removing Wipes',
    'Makeup Removing Cleansing Balm with Jojoba Oil for All Skin Types': 'CeraVe Makeup Removing Cleansing Balm'

}
Cerave_df['Name'] = Cerave_df['Name'].replace(cerave_mapping)
print(Cerave_df['Name'].unique())
df.update(Cerave_df)

['CeraVe Resurfacing Retinol Serum' 'CeraVe Daily Moisturizing Lotion'
 'CeraVe Hydrating Facial Cleanser'
 'CeraVe Hydrating Cream-to-Foam Cleanser' 'CeraVe Renewing SA Cleanser'
 'CeraVe Foaming Facial Cleanser'
 'CeraVe Hydrating Cleanser for Normal to Dry Skin USA'
 'CeraVe Hydrating Foaming Oil Cleanser'
 'CeraVe Skin Renewing Night Cream' 'CeraVe Moisturizing Cream'
 'CeraVe PM Lotion' 'CeraVe AM Lotion SPF 30'
 'CeraVe Skin Renewing Vitamin C Serum' 'CeraVe Skin Renewing Eye Cream'
 'CeraVe Acne Control Cleanser' 'CeraVe Skin Renewing Retinol Serum'
 'CeraVe Hydrating Hyaluronic Acid Serum' 'CeraVe SA Cream'
 'Eye Repair Cream for Dark Circles and Puffiness for All Skin Types'
 'CeraVe Acne Foaming Cream Cleanser'
 'CeraVe Ultra-Light Moisturizing Lotion'
 'Alcohol-Free No Rinse Hydrating Facial Toner for Sensitive & Dry Skin'
 'CeraVe Makeup Removing Wipes' 'CeraVe Mineral Sunscreen SPF 30'
 'CeraVe Healing Ointment' 'CeraVe Ultra-Light Gel'
 'CeraVe Acne Foaming Cream Wash' 'C

C:\Users\Hobopk\AppData\Local\Temp/ipykernel_16204/898500858.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cerave_df['Name'] = Cerave_df['Name'].replace(cerave_mapping)


In [14]:
Ordinary_df = df[df['Brand'] == "The Ordinary"]
Ordinary_products = Ordinary_df['Name'].unique()
#print(Ordinary_products)
ordinary_mapping = {
    
    'The Ordinary Niacinamide 10% + Zinc 1% Serum': 'The Ordinary Niacinamide 10% + Zinc 1% Serum',
    'The Ordinary Salicylic Acid 2% Solution - 30ml': 'The Ordinary Salicylic Acid 2% Solution',
    'The Ordinary Retinol 1% In Squalane - 30ml': 'The Ordinary Retinol 1% In Squalane',
    'The Ordinary Glycolic Acid 7% Toning Solution - 240ml': 'The Ordinary Glycolic Acid 7% Toning Solution',
    'The Ordinary Retinol 0.5% In Squalane - 30ml': 'The Ordinary Retinol 0.5% In Squalane',
    'The Ordinary Vitamin C Suspension 23% + HA Spheres 2% Size - 30ml': 'The Ordinary Vitamin C Suspension 23% + HA Spheres 2%',
    'The Ordinary Retinol 0.2% In Squalane - 30ml': 'The Ordinary Retinol 0.2% In Squalane',
    'The Ordinary Hyaluronic Acid 2% + B5': 'The Ordinary Hyaluronic Acid 2% + B5',
    'The Ordinary Vitamin C Suspension 30% In Silicone - 30ml': 'The Ordinary Vitamin C Suspension 23% + HA Spheres 2%',
    'The Ordinary Lactic Acid 10% + HA 2% Volume - 30ml': 'The Ordinary Lactic Acid 10% + HA 2% Volume',
    'The Ordinary Alpha Arbutin 2% + HA - 30ml': 'The Ordinary Alpha Arbutin 2% + HA',
    'The Ordinary- Hyaluronic Acid 2% + B5': 'The Ordinary Hyaluronic Acid 2% + B5',
    'The Ordinary- Retinol 1% In Squalane 30Ml': 'The Ordinary Retinol 1% In Squalane',
    'The Ordinary- Niacinamide 10% + Zinc 1% 30Ml': 'The Ordinary- Niacinamide 10% + Zinc 1%',
    'The Ordinary Alpha Arbutin 2% + HA': 'The Ordinary Hyaluronic Acid 2% + B5',
    'The Ordinary Lactic Acid 5% + HA': 'The Ordinary Lactic Acid 5% + HA',
    'THE ORDINARY Niacinamide 10% + Zinc 1%': 'The Ordinary Niacinamide 10% + Zinc 1% Serum',
    'The Ordinary Salicylic Acid 2% Solution': 'The Ordinary Salicylic Acid 2% Solution',
    'THE ORDINARY - NIACINAMIDE 10% + Zinc 1%': 'The Ordinary Niacinamide 10% + Zinc 1% Serum',
    'The Ordinary Lactic Acid 10% + HA 2%': 'The Ordinary Lactic Acid 10% + HA 2% Volume',
    'TO Niacinamide 10% + Zinc 1% (60ml)': 'The Ordinary Niacinamide 10% + Zinc 1% Serum',
    'TO Niacinamide Serum': 'The Ordinary Niacinamide 10% + Zinc 1% Serum',
    'TO Alpha Arbutin 2% + HA': 'The Ordinary Hyaluronic Acid 2% + B5',
    'TO AHA 30% + BHA 2% Peeling Solution': 'AHA 30% + BHA 2% Exfoliating Peeling Solution',
    'TO Niacinamide 10% + Zinc 1% (30ml)': 'The Ordinary Niacinamide 10% + Zinc 1% Serum',
    'TO Glycolic Acid 7% Toning Solution': 'The Ordinary Glycolic Acid 7% Toning Solution',
    'TO Hyaluronic Acid 2% + B5 30ml': 'The Ordinary Hyaluronic Acid 2% + B5',
    'Glycolic Acid 7% Exfoliating Toner': 'The Ordinary Glycolic Acid 7% Toning Solution',
    'Hyaluronic Acid 2% + B5 Hydrating Serum with Ceramides': 'Hyaluronic Acid 2% + B5 Hydrating Serum with Ceramides',
    'Natural Moisturizing Factors + Hyaluronic Acid Daily Moisturizer': 'Natural Moisturizing Factors + Hyaluronic Acid Daily Moisturizer',
    'The Most-Loved Skin Care Mini Holiday Gift Set': 'Skin Care Set',
    'The Acne Set 2.0: 3-Step Regimen': 'Acne Skin Care Set',
    'Multi-Peptide Serum for Hair Density': 'Multi-Peptide Serum for Hair Density',
    'Azelaic Acid 10% Suspension Brightening Cream': 'Azelaic Acid 10% Suspension Brightening Cream',
    'Caffeine 5% + EGCG Brightening Eye Serum': 'Caffeine 5% + EGCG Brightening Eye Serum',
    'Multi-Peptide + Copper Peptides 1% Serum For Aging Skin': 'Multi-Peptide + Copper Peptides 1% Serum For Aging Skin',
    'Argireline 10% Serum for Facial Wrinkles': 'Argireline 10% Serum for Facial Wrinkles',
    'Salicylic Acid 2% Acne Serum': 'The Ordinary Salicylic Acid 2% Solution',
    'Travel Size Glycolic Acid 7% Exfoliating Toner': 'The Ordinary Glycolic Acid 7% Toning Solution',
    'Multi-Peptide + Hyaluronic Acid Serum For Aging Skin': 'Anti-Aging Peptide Serum with Hydration',
    'Squalane Cleanser For Dry Skin': 'Squalane Cleanser For Dry Skin',
    'Soothing & Barrier Support Serum': 'Soothing & Barrier Support Serum',
    'Retinol 0.5% in Squalane Serum': 'The Ordinary Retinol 0.5% In Squalane',
    'The Bright Skin Care Holiday Gift Set': 'Skin Care Set',
    'Glucoside Foaming Facial Cleanser': 'Glucoside Foaming Facial Cleanser',
    '100% Organic Cold-Pressed Rose Hip Seed Oil for Aging Skin': '100% Organic Cold-Pressed Rose Hip Seed Oil for Aging Skin',
    'Matrixyl 10% + HA High-Strength Wrinkle Support Serum': 'Matrixyl 10% + HA High-Strength Wrinkle Support Serum',
    'Multi-Peptide Anti-Aging Eye Serum': 'Multi-Peptide Anti-Aging Eye Serum',
    'Saccharomyces Ferment 30% Milky Toner': 'Saccharomyces Ferment 30% Milky Toner',
    'Natural Moisturizing Factors + PhytoCeramides Rich Moisturizer': 'Natural Moisturizing Factors + PhytoCeramides Rich Moisturizer',
    '100% Plant-Derived Squalane Skin and Hair Hydrator Serum': '100% Plant-Derived Squalane Skin and Hair Hydrator Serum',
    'Granactive Retinoid 2% Emulsion': 'Anti-Aging Retinoid Emulsion',
    'Balancing & Clarifying Serum for Blemish Prone Skin': 'Balancing Serum for Blemish Prone Skin',
    'Salicylic Acid 2% Mask for Blemish Prone Skin': 'Salicylic Acid 2% Mask for Blemish Prone Skin',
    'The Power of Peptides Skin Care Set': 'Peptide Skin Care Set',
    'Retinal 0.2% Emulsion': 'The Ordinary Retinol 0.2% In Squalane',
    'Vitamin C Suspension 23% + HA Spheres 2% Serum': 'Vitamin C Suspension 23% + HA Spheres 2% Serum',
    'Lactic Acid 10% + HA 2% Exfoliating Serum': 'Lactic Acid 10% + HA 2% Exfoliating Serum',
    'Natural Moisturizing Factors + Beta Glucan Gel Moisturizer': 'Natural Moisturizing Factors + Beta Glucan Gel Moisturizer',
    'Ascorbyl Glucoside Solution 12% Serum': 'Ascorbyl Glucoside Solution 12% Serum',
    '100% Cold Pressed Virgin Marula Oil': '100% Cold Pressed Virgin Marula Oil',
    'Glycolipid Cream Facial Cleanser': 'Glycolipid Cream Facial Cleanser',
    'Niacinamide 5% Face and Body Emulsion': 'Niacinamide 5% Face and Body Emulsion',
    'Salicylic Acid 2% Anhydrous Solution Serum': 'Salicylic Acid 2% Anhydrous Solution Serum',
    'Marine Hyaluronic Moisture Lock Serum': 'Marine Hyaluronic Moisture Lock Serum',
    'Granactive Retinoid 5% in Squalane Serum': 'Granactive Retinoid 5% in Squalane Serum',
    '100% Organic Cold-Pressed Moroccan Argan Oil': '100% Organic Cold-Pressed Moroccan Argan Oil',
    'Aloe 2% + NAG 2% Solution Serum': 'Aloe 2% + NAG 2% Solution Serum',
    'Natural Moisturizing Factors + Hyaluronic Acid Scalp Serum': 'Natural Moisturizing Factors + Hyaluronic Acid Scalp Serum',
    'Salicylic Acid 0.5% Body Serum': 'Salicylic Acid 0.5% Body Serum',
    'Pycnogenol 5% High-Potency Antioxidant Serum': 'Pycnogenol 5% High-Potency Antioxidant Serum',
    'Amino Acids + B5 Hydrating Serum': 'Amino Acids + B5 Hydrating Serum',
    'Resveratrol 3% + Ferulic Acid 3% Serum': 'Resveratrol 3% + Ferulic Acid 3% Serum',
    '100% Niacinamide High-Potency Oil-Control Powder': '100% Niacinamide High-Potency Oil-Control Powder',
    'Travel Size AHA 30% + BHA 2% Exfoliating Peeling Solution': 'AHA 30% + BHA 2% Exfoliating Peeling Solution',

}
Ordinary_df['Name'] = Ordinary_df['Name'].replace(ordinary_mapping)
print(Ordinary_df['Name'].unique())
df.update(Ordinary_df)

['The Ordinary Niacinamide 10% + Zinc 1% Serum'
 'The Ordinary Salicylic Acid 2% Solution'
 'The Ordinary Retinol 1% In Squalane'
 'The Ordinary Glycolic Acid 7% Toning Solution'
 'The Ordinary Retinol 0.5% In Squalane'
 'The Ordinary Vitamin C Suspension 23% + HA Spheres 2%'
 'The Ordinary Retinol 0.2% In Squalane'
 'The Ordinary Hyaluronic Acid 2% + B5'
 'The Ordinary Lactic Acid 10% + HA 2% Volume'
 'The Ordinary Alpha Arbutin 2% + HA'
 'The Ordinary- Niacinamide 10% + Zinc 1%'
 'The Ordinary Lactic Acid 5% + HA'
 'AHA 30% + BHA 2% Exfoliating Peeling Solution'
 'Hyaluronic Acid 2% + B5 Hydrating Serum with Ceramides'
 'Natural Moisturizing Factors + Hyaluronic Acid Daily Moisturizer'
 'Skin Care Set' 'Acne Skin Care Set'
 'Multi-Peptide Serum for Hair Density'
 'Azelaic Acid 10% Suspension Brightening Cream'
 'Caffeine 5% + EGCG Brightening Eye Serum'
 'Multi-Peptide + Copper Peptides 1% Serum For Aging Skin'
 'Argireline 10% Serum for Facial Wrinkles'
 'Anti-Aging Peptide Serum wi

C:\Users\Hobopk\AppData\Local\Temp/ipykernel_16204/2313032196.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ordinary_df['Name'] = Ordinary_df['Name'].replace(ordinary_mapping)


In [15]:
# Filter the Neutrogena products only
Neutrogena_df = df[df['Brand'] == "Neutrogena"]

# Remove CeraVe products
Neutrogena_df = Neutrogena_df[~Neutrogena_df['Name'].str.contains('CeraVe', na=False)]

# Get unique Neutrogena products
Neutrogena_products = Neutrogena_df['Name'].unique()

# Print the Neutrogena products
#print(Neutrogena_products)
neutrogena_mapping = {
    'Neutrogena Hydro Boost Water Gel': 'Neutrogena Hydro Boost Cleanser Water Gel',
    'Neutrogena Oil Free Acne Wash': 'Neutrogena Oil Free Acne Wash',
    'Neutrogena Daily Lip Balm SPF 15': 'Neutrogena Daily Lip Balm SPF 15',
    'Neutrogena Spot Controlling Facial Wash 200Ml': 'Neutrogena Spot Controlling Facial Wash 200Ml',
    'Neutrogena spot controlling facial wash 200ml': 'Neutrogena Spot Controlling Facial Wash 200Ml',
    'Neutrogena Oil-Free Acne Wash 269Ml': 'Neutrogena Oil Free Acne Wash',
    'Neutrogena Hydra Boost Hyaluronic Acid Serum': 'Neutrogena Hydra Boost Hyaluronic Acid Serum',
    'Neutrogena Hydro Boost Water Gel Cleanser 200ML': 'Neutrogena Hydro Boost Cleanser Water Gel',
    'Neutrogena Ultra Sheer Dry-Touch Sunblock SPF 50+ PA++++': 'Neutrogena Ultra Sheer Dry-Touch Sunblock SPF 50+ PA++++',
    'Neutrogena Facial Wash Oil Balancing With Lime & Aloe Vera Pump 200Ml': 'Neutrogena Facial Wash Oil Balancing With Lime & Aloe Vera Pump 200Ml',
    'Neutrogena Facial Wash Fresh & Clear With Pink Grapefruit Pump 200M (Co)': 'Neutrogena Facial Wash Fresh & Clear With Pink Grapefruit Pump 200Ml',
    'Neutrogena Hydro Boost Cleanser Water Gel': 'Neutrogena Hydro Boost Cleanser Water Gel',
    'Neutrogena Ultra Sheer® Dry-Touch Sunscreen Broad Spectrum SPF 45 ( Expiry-3-2024 )': 'Neutrogena Ultra Sheer® Dry-Touch Sunscreen Broad Spectrum SPF 45 ( Expiry-3-2024 )',
    'Hydro Boost Hyaluronic Acid Water Gel Moisturizer': 'Hydro Boost Hyaluronic Acid Water Gel Moisturizer',
    'Hydro Boost Hydrating 100% Hydrogel Mask': 'Hydro Boost Hydrating 100% Hydrogel Mask'
}

Neutrogena_df['Name'] = Neutrogena_df['Name'].replace(neutrogena_mapping)
print(Neutrogena_df['Name'].unique())
df.update(Neutrogena_df)


['Neutrogena Hydro Boost Cleanser Water Gel'
 'Neutrogena Oil Free Acne Wash' 'Neutrogena Daily Lip Balm SPF 15'
 'Neutrogena Spot Controlling Facial Wash 200Ml'
 'Neutrogena Hydra Boost Hyaluronic Acid Serum'
 'Neutrogena Ultra Sheer Dry-Touch Sunblock SPF 50+ PA++++'
 'Neutrogena Facial Wash Oil Balancing With Lime & Aloe Vera Pump 200Ml'
 'Neutrogena Facial Wash Fresh & Clear With Pink Grapefruit Pump 200Ml'
 'Neutrogena Ultra Sheer® Dry-Touch Sunscreen Broad Spectrum SPF 45 ( Expiry-3-2024 )'
 'Hydro Boost Hyaluronic Acid Water Gel Moisturizer'
 'Hydro Boost Hydrating 100% Hydrogel Mask']


In [16]:
bodyshop_df = df[df['Brand'] == "The Body Shop"]
bodyshop_products = bodyshop_df['Name'].unique()
bodyshop_mapping = {
    'The Body Shop Seaweed Oil-Control Face Wash 125ml': 'The Body Shop Seaweed Oil-Control Face Wash',
    'The Body Shop Body Yogurt - Almond Milk': 'The Body Shop Body Yogurt - Almond Milk',
    'The Body Shop Himalayan Charcoal 15Ml': 'The Body Shop Himalayan Charcoal',
    'The Body Shop Himalayan Charcoal 75Ml': 'The Body Shop Himalayan Charcoal',
    'The Body Shop Vitamin C Liquid Peel': 'The Body Shop Vitamin C Liquid Peel',
    'The Body Shop Vitamin C Glow Boosting Moisturiser': 'The Body Shop Vitamin C Glow Boosting Moisturiser',
    'The Body Shop Drops of Youth™ Concentrate': 'The Body Shop Drops of Youth™ Concentrate',
    'The Body Shop Vitamin E Gentle Facial Wash 125ml': 'The Body Shop Vitamin E Gentle Facial Wash',
    'The Body Shop Oils of Life™ Sleeping Cream': 'The Body Shop Oils of Life™ Sleeping Cream',
    'The Body Shop Tea Tree Cleasing Facial Wash 60Ml': 'The Body Shop Tea Tree Cleansing Facial Wash',
    'The Body Shop Tea Tree Skin Clearing Facial Wash 250ml': 'The Body Shop Tea Tree Cleansing Facial Wash',
    'The Body Shop Tea Tree Squeaky Clean Scrub 125ml': 'The Body Shop Tea Tree Squeaky Clean Scrub',
    'The Body Shop Vitamin E Overnight Serum-In-Oil': 'The Body Shop Vitamin E Overnight Serum-In-Oil',
    'The Body Shop Tea Tree Anti-Imperfection Daily Solution': 'The Body Shop Tea Tree Anti-Imperfection Daily Solution',
    'The Body Shop Vitamin C Daily Glow Cleansing Polish 100ml': 'The Body Shop Vitamin C Daily Glow Cleansing Polish',
    'The Body Shop Tea Tree Skin Clearing Night Mask 75Ml': 'The Body Shop Tea Tree Skin Clearing Night Mask',
    'The Body Shop Chinese Ginseng and Rice Clarifying Polishing Mask': 'The Body Shop Chinese Ginseng and Rice Clarifying Polishing Mask',
    'The Body Shop Drops Of Light™ Brightening Day Cream': 'The Body Shop Drops Of Light™ Brightening Day Cream',
    'The Body Shop Tea Tree Skin Clearing Mattifying Toner - 250 Ml': 'The Body Shop Tea Tree Skin Clearing Mattifying Toner',
    'The Body Shop Tea Tree Mattifying Toner 60Ml': 'The Body Shop Tea Tree Skin Clearing Mattifying Toner',
    'The Body Shop Skin Defence Multi-Protection Lotion SPF 50+ PA++++': 'The Body Shop Skin Defence Multi-Protection Lotion SPF 50+ PA++++',
    'The Body Shop Drops of Light™ Brightening Pure Clarifying Foam Wash': 'The Body Shop Drops of Light™ Brightening Pure Clarifying Foam Wash',
    'The Crème Shop x Hello Kitty Fusion Body Scrub - Caramel Pumpkin Latte': 'The Crème Shop x Hello Kitty Fusion Body Scrub - Caramel Pumpkin Latte',
    'The Body Shop Eyeliner Liquid - 01 Black': 'The Body Shop Eyeliner Liquid - 01 Black',
    'Coconut Sugar Body Scrub': 'Coconut Sugar Body Scrub'
}
bodyshop_df['Name'] = bodyshop_df['Name'].replace(bodyshop_mapping)
print(bodyshop_df['Name'].unique())
df.update(bodyshop_df)

['The Body Shop Seaweed Oil-Control Face Wash'
 'The Body Shop Body Yogurt - Almond Milk'
 'The Body Shop Himalayan Charcoal' 'The Body Shop Vitamin C Liquid Peel'
 'The Body Shop Vitamin C Glow Boosting Moisturiser'
 'The Body Shop Drops of Youth™ Concentrate'
 'The Body Shop Vitamin E Gentle Facial Wash'
 'The Body Shop Oils of Life™ Sleeping Cream'
 'The Body Shop Tea Tree Cleansing Facial Wash'
 'The Body Shop Tea Tree Squeaky Clean Scrub'
 'The Body Shop Vitamin E Overnight Serum-In-Oil'
 'The Body Shop Tea Tree Anti-Imperfection Daily Solution'
 'The Body Shop Vitamin C Daily Glow Cleansing Polish'
 'The Body Shop Tea Tree Skin Clearing Night Mask'
 'The Body Shop Chinese Ginseng and Rice Clarifying Polishing Mask'
 'The Body Shop Drops Of Light™ Brightening Day Cream'
 'The Body Shop Tea Tree Skin Clearing Mattifying Toner'
 'The Body Shop Skin Defence Multi-Protection Lotion SPF 50+ PA++++'
 'The Body Shop Drops of Light™ Brightening Pure Clarifying Foam Wash'
 'The Crème Shop 

C:\Users\Hobopk\AppData\Local\Temp/ipykernel_16204/2607923868.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bodyshop_df['Name'] = bodyshop_df['Name'].replace(bodyshop_mapping)


In [18]:
# Adding features
skin_type_mapping = {
    'Conatural Hair Conditioner': 'All/Hair Care',
    'Conatural Tea Tree Essential Oil': 'Oily/Acne-prone',
    'Conatural Saffron Complexion Builder': 'All/Dry/Hyperpigmentation',
    'Conatural Anti-Aging - Super Activs Skin Serum': 'All/Aging',
    'Conatural Anti-Pigmentation - Super Activs Skin Serum': 'Pigmentation',
    'Conatural Hyaluronic Acid 2% + B5 - Super Activs Skin Serum': 'Dry/Sensitive',
    'Conatural Niacinamide 10% + Zinc 1% - Super Activs Skin Serum': 'Oily/Acne-prone',
    'Conatural Rose Face Wash': 'Normal/Dry/Sensitive',
    'The Moms Co. Natural Vita Rich Face Serum with Vitamins and Hyaluronic Acid': 'All/Dry',
    'The Moms Co Natural Vita Rich Face Serum (8 ml) (Mini / Small ...': 'All/Dry',
    'The Moms Co. Natural Baby Lotion with Rice Bran, Apricot, Jojoba Oil, Cocoa and Shea Butter': 'Baby/Sensitive',
    'The Body Shop Seaweed Oil-Control Face Wash': 'Combination/Oily',
    'The Body Shop Body Yogurt - Almond Milk': 'Sensitive/Dry',
    'The Body Shop Himalayan Charcoal': 'Oily/Combination',
    'The Body Shop Vitamin C Liquid Peel': 'All',
    'The Body Shop Vitamin C Glow Boosting Moisturiser': 'All',
    'The Body Shop Drops of Youth™ Concentrate': 'All',
    'The Body Shop Vitamin E Gentle Facial Wash': 'Normal/Dry',
    'The Body Shop Oils of Life™ Sleeping Cream': 'Dry',
    'The Body Shop Tea Tree Cleansing Facial Wash': 'Oily/Acne-prone',
    'The Body Shop Tea Tree Squeaky Clean Scrub': 'Oily/Acne-prone',
    'The Body Shop Vitamin E Overnight Serum-In-Oil': 'Dry',
    'The Body Shop Tea Tree Anti-Imperfection Daily Solution': 'Oily/Acne-prone',
    'The Body Shop Vitamin C Daily Glow Cleansing Polish': 'All',
    'The Body Shop Tea Tree Skin Clearing Night Mask': 'Oily/Acne-prone',
    'The Body Shop Chinese Ginseng and Rice Clarifying Polishing Mask': 'All',
    'The Body Shop Drops Of Light™ Brightening Day Cream': 'All',
    'The Body Shop Tea Tree Skin Clearing Mattifying Toner': 'Oily/Acne-prone',
    'The Body Shop Skin Defence Multi-Protection Lotion SPF 50+ PA++++': 'All',
    'The Body Shop Drops of Light™ Brightening Pure Clarifying Foam Wash': 'All',
    'The Crème Shop x Hello Kitty Fusion Body Scrub - Caramel Pumpkin Latte': 'All',
    'The Body Shop Eyeliner Liquid - 01 Black': 'All',
    'Coconut Sugar Body Scrub': 'All',
    'Neutrogena Hydro Boost Cleanser Water Gel': 'All',
    'Neutrogena Oil Free Acne Wash': 'Oily/Acne-Prone',
    'Neutrogena Daily Lip Balm SPF 15': 'All',
    'Neutrogena Spot Controlling Facial Wash 200Ml': 'Acne-Prone',
    'Neutrogena Hydra Boost Hyaluronic Acid Serum': 'All',
    'Neutrogena Ultra Sheer Dry-Touch Sunblock SPF 50+ PA++++': 'All',
    'Neutrogena Facial Wash Oil Balancing With Lime & Aloe Vera Pump 200Ml': 'Oily',
    'Neutrogena Facial Wash Fresh & Clear With Pink Grapefruit Pump 200Ml': 'Combination/Acne-Prone',
    'Neutrogena Ultra Sheer® Dry-Touch Sunscreen Broad Spectrum SPF 45 ( Expiry-3-2024 )': 'All',
    'Hydro Boost Hyaluronic Acid Water Gel Moisturizer': 'All',
    'Hydro Boost Hydrating 100% Hydrogel Mask': 'All',
    'The Ordinary Niacinamide 10% + Zinc 1% Serum': 'All',
    'The Ordinary Salicylic Acid 2% Solution': 'Oily/Acne-Prone',
    'The Ordinary Retinol 1% In Squalane': 'All',
    'The Ordinary Glycolic Acid 7% Toning Solution': 'All',
    'The Ordinary Retinol 0.5% In Squalane': 'All',
    'The Ordinary Vitamin C Suspension 23% + HA Spheres 2%': 'All',
    'The Ordinary Retinol 0.2% In Squalane': 'All',
    'The Ordinary Hyaluronic Acid 2% + B5': 'All',
    'The Ordinary Lactic Acid 10% + HA 2% Volume': 'All',
    'The Ordinary- Niacinamide 10% + Zinc 1%': 'All',
    'The Ordinary Lactic Acid 5% + HA': 'All',
    'AHA 30% + BHA 2% Exfoliating Peeling Solution': 'All/Acne-Prone',
    'Hyaluronic Acid 2% + B5 Hydrating Serum with Ceramides': 'All',
    'Natural Moisturizing Factors + Hyaluronic Acid Daily Moisturizer': 'All',
    'Skin Care Set': 'All',
    'Acne Skin Care Set': 'Oily/Acne-Prone',
    'Multi-Peptide Serum for Hair Density': 'All',
    'Brightening Cream': 'All',
    'Eye Brightening Serum': 'All',
    'Anti-Aging Peptide Serum': 'All',
    'Anti-Wrinkle Serum': 'All',
    'Anti-Aging Peptide Serum with Hydration': 'All',
    'Squalane Cleanser For Dry Skin': 'Dry',
    'Soothing & Barrier Support Serum': 'Sensitive/Dry',
    'Glucoside Foaming Facial Cleanser': 'All',
    '100% Organic Cold-Pressed Rose Hip Seed Oil for Aging Skin': 'Dry/Aging',
    'Matrixyl 10% + HA High-Strength Wrinkle Support Serum': 'All/Aging',
    'Multi-Peptide Anti-Aging Eye Serum': 'All',
    'Saccharomyces Ferment 30% Milky Toner': 'All',
    'Natural Moisturizing Factors + PhytoCeramides Rich Moisturizer': 'Dry',
    '100% Plant-Derived Squalane Skin and Hair Hydrator Serum': 'All/Dry',
    'Anti-Aging Retinoid Emulsion': 'All/Aging',
    'Balancing Serum for Blemish Prone Skin': 'Oily/Acne-Prone',
    'Salicylic Acid 2% Mask for Blemish Prone Skin': 'Oily/Acne-Prone',
    'Peptide Skin Care Set': 'All',
    'Vitamin C Suspension 23% + HA Spheres 2% Serum': 'All',
    'Lactic Acid 10% + HA 2% Exfoliating Serum': 'All',
    'Natural Moisturizing Factors + Beta Glucan Gel Moisturizer': 'All',
    'Ascorbyl Glucoside Solution 12% Serum': 'All',
    '100% Cold Pressed Virgin Marula Oil': 'Dry',
    'Glycolipid Cream Facial Cleanser': 'Dry/Sensitive',
    'Niacinamide 5% Face and Body Emulsion': 'All',
    'Salicylic Acid 2% Anhydrous Solution Serum': 'Oily/Acne-Prone',
    'Marine Hyaluronic Moisture Lock Serum': 'All',
    'Granactive Retinoid 5% in Squalane Serum': 'All',
    '100% Organic Cold-Pressed Moroccan Argan Oil': 'Dry',
    'Aloe 2% + NAG 2% Solution Serum': 'Sensitive/All',
    'Natural Moisturizing Factors + Hyaluronic Acid Scalp Serum': 'All',
    'Salicylic Acid 0.5% Body Serum': 'Oily/Acne-Prone',
    'Pycnogenol 5% High-Potency Antioxidant Serum': 'All',
    'Amino Acids + B5 Hydrating Serum': 'All',
    'Resveratrol 3% + Ferulic Acid 3% Serum': 'All',
    '100% Niacinamide High-Potency Oil-Control Powder': 'Oily',
    'Cosrx Acne Pimple Master Patch': 'All/Acne-prone',
    'COSRX AHA/BHA Clarifying Treatment Toner': 'All',
    'Cosrx One Step Original Clear Pad': 'All/Acne-prone',
    'Cosrx Clear Fit Master Patch': 'All/Acne-prone',
    'Cosrx BHA Blackhead Power Liquid': 'All/Oily/Acne-prone',
    'COSRX The Vitamin C 23 serum 20ml': 'All',
    'Cosrx Advanced Snail 96 Mucin Power Essence': 'All/Dry/Sensitive',
    'Cosrx Salicylic Acid Daily Gentle Cleanser': 'All/Oily/Acne-prone',
    'COSRX - OIL-FREE ULTRA-MOISTURIZING LOTION 100ml': 'All',
    'Cosrx Low pH Good Morning Gel Cleanser': 'All/Sensitive',
    'COSRX - Aloe Soothing Sun Cream SPF50+/ PA+++ 50ml': 'All',
    'Advanced Snail 92 All In One Cream': 'All/Dry/Sensitive',
    'CeraVe Resurfacing Retinol Serum': 'All/Oily/Acne-prone',
    'CeraVe Daily Moisturizing Lotion': 'All/Normal/Dry',
    'CeraVe Hydrating Facial Cleanser': 'Normal/Dry/Sensitive',
    'CeraVe Hydrating Cream-to-Foam Cleanser': 'Normal/Dry/Sensitive',
    'CeraVe Renewing SA Cleanser': 'Oily/Acne-prone',
    'CeraVe Foaming Facial Cleanser': 'Normal/Oily',
    'CeraVe Hydrating Cleanser for Normal to Dry Skin USA': 'Normal/Dry',
    'CeraVe Hydrating Foaming Oil Cleanser': 'Normal/Dry',
    'CeraVe Skin Renewing Night Cream': 'Normal/Dry',
    'CeraVe Moisturizing Cream': 'Normal/Dry/Sensitive',
    'CeraVe PM Lotion': 'Normal/Dry/Sensitive',
    'CeraVe AM Lotion SPF 30': 'Normal/Dry/Sensitive',
    'CeraVe Skin Renewing Vitamin C Serum': 'All/Sensitive',
    'CeraVe Skin Renewing Eye Cream': 'All',
    'CeraVe Acne Control Cleanser': 'Oily/Acne-prone',
    'CeraVe Skin Renewing Retinol Serum': 'All',
    'CeraVe Hydrating Hyaluronic Acid Serum': 'All/Dry/Sensitive',
    'CeraVe SA Cream': 'Oily/Acne-prone/Rough',
    'Eye Repair Cream for Dark Circles and Puffiness for All Skin Types': 'All',
    'CeraVe Acne Foaming Cream Cleanser': 'Oily/Acne-prone',
    'CeraVe Ultra-Light Moisturizing Lotion': 'Oily',
    'Alcohol-Free No Rinse Hydrating Facial Toner for Sensitive & Dry Skin': 'Sensitive/Dry',
    'CeraVe Makeup Removing Wipes': 'All/Sensitive',
    'CeraVe Mineral Sunscreen SPF 30': 'All',
    'CeraVe Healing Ointment': 'All/Dry',
    'CeraVe Ultra-Light Gel': 'Oily/Acne-prone',
    'CeraVe Acne Foaming Cream Wash': 'Oily/Acne-prone',
    'CeraVe Makeup Removing Cleansing Balm': 'All/Sensitive',
    'CeraVe SA Lotion': 'Oily/Acne-prone/Rough',
    'CeraVe Skin Renewing Nightly Exfoliating Treatment': 'All',
    'CeraVe Hydrating Micellar Water': 'All/Sensitive',
    'CeraVe Skin Renewing Gel Oil': 'Dry',
    'CeraVe Skin Renewing Day Cream': 'Dry',
    'Travel Size Daily Moisturizing Lotion for Balanced to Dry Skin': 'Normal/Dry',
    'CeraVe AM Lotion SPF 50': 'Normal/Dry/Sensitive',
    'CeraVe Acne Control Gel': 'Oily/Acne-prone',
    'CeraVe Comforting Eye Makeup Remover': 'All/Sensitive',
    'CeraVe Mineral Sunscreen SPF 50': 'All',
    'CeraVe Oil Control Gel-Cream': 'Oily',
    'CeraVe Sheer Sunscreen SPF 30': 'All'
}

category_mapping = {
    'Conatural Hair Conditioner': 'Hair Care',
    'Conatural Tea Tree Essential Oil': 'Essential Oil',
    'Conatural Saffron Complexion Builder': 'Serum',
    'Conatural Anti-Aging - Super Activs Skin Serum': 'Serum',
    'Conatural Anti-Pigmentation - Super Activs Skin Serum': 'Serum',
    'Conatural Hyaluronic Acid 2% + B5 - Super Activs Skin Serum': 'Serum',
    'Conatural Niacinamide 10% + Zinc 1% - Super Activs Skin Serum': 'Serum',
    'Conatural Rose Face Wash': 'Cleanser',
    'The Moms Co. Natural Vita Rich Face Serum with Vitamins and Hyaluronic Acid': 'Serum',
    'The Moms Co Natural Vita Rich Face Serum (8 ml) (Mini / Small ...)': 'Serum',
    'The Moms Co. Natural Baby Lotion with Rice Bran, Apricot, Jojoba Oil, Cocoa and Shea Butter': 'Baby Care/Lotion',
    'The Body Shop Seaweed Oil-Control Face Wash': 'Cleanser',
    'The Body Shop Body Yogurt - Almond Milk': 'Moisturizer',
    'The Body Shop Himalayan Charcoal': 'Serum',
    'The Body Shop Vitamin C Liquid Peel': 'Exfoliator',
    'The Body Shop Vitamin C Glow Boosting Moisturiser': 'Moisturizer',
    'The Body Shop Drops of Youth™ Concentrate': 'Serum',
    'The Body Shop Vitamin E Gentle Facial Wash': 'Cleanser',
    'The Body Shop Oils of Life™ Sleeping Cream': 'Night Cream',
    'The Body Shop Tea Tree Cleansing Facial Wash': 'Cleanser',
    'The Body Shop Tea Tree Squeaky Clean Scrub': 'Scrub',
    'The Body Shop Vitamin E Overnight Serum-In-Oil': 'Serum',
    'The Body Shop Tea Tree Anti-Imperfection Daily Solution': 'Serum',
    'The Body Shop Vitamin C Daily Glow Cleansing Polish': 'Cleanser',
    'The Body Shop Tea Tree Skin Clearing Night Mask': 'Mask',
    'The Body Shop Chinese Ginseng and Rice Clarifying Polishing Mask': 'Mask',
    'The Body Shop Drops Of Light™ Brightening Day Cream': 'Moisturizer',
    'The Body Shop Tea Tree Skin Clearing Mattifying Toner': 'Toner',
    'The Body Shop Skin Defence Multi-Protection Lotion SPF 50+ PA++++': 'Sunscreen',
    'The Body Shop Drops of Light™ Brightening Pure Clarifying Foam Wash': 'Cleanser',
    'The Crème Shop x Hello Kitty Fusion Body Scrub - Caramel Pumpkin Latte': 'Scrub',
    'The Body Shop Eyeliner Liquid - 01 Black': 'Makeup',
    'Coconut Sugar Body Scrub': 'Scrub',
    'Neutrogena Hydro Boost Cleanser Water Gel': 'Cleanser',
    'Neutrogena Oil Free Acne Wash': 'Cleanser',
    'Neutrogena Daily Lip Balm SPF 15': 'Lip Care',
    'Neutrogena Spot Controlling Facial Wash 200Ml': 'Cleanser',
    'Neutrogena Hydra Boost Hyaluronic Acid Serum': 'Serum',
    'Neutrogena Ultra Sheer Dry-Touch Sunblock SPF 50+ PA++++': 'Sunscreen',
    'Neutrogena Facial Wash Oil Balancing With Lime & Aloe Vera Pump 200Ml': 'Cleanser',
    'Neutrogena Facial Wash Fresh & Clear With Pink Grapefruit Pump 200Ml': 'Cleanser',
    'Neutrogena Ultra Sheer® Dry-Touch Sunscreen Broad Spectrum SPF 45 ( Expiry-3-2024 )': 'Sunscreen',
    'Hydro Boost Hyaluronic Acid Water Gel Moisturizer': 'Moisturizer',
    'Hydro Boost Hydrating 100% Hydrogel Mask': 'Mask',
     'The Ordinary Niacinamide 10% + Zinc 1% Serum': 'Serum',
    'The Ordinary Salicylic Acid 2% Solution': 'Exfoliator',
    'The Ordinary Retinol 1% In Squalane': 'Serum',
    'The Ordinary Glycolic Acid 7% Toning Solution': 'Toner',
    'The Ordinary Retinol 0.5% In Squalane': 'Serum',
    'The Ordinary Vitamin C Suspension 23% + HA Spheres 2%': 'Serum',
    'The Ordinary Retinol 0.2% In Squalane': 'Serum',
    'The Ordinary Hyaluronic Acid 2% + B5': 'Serum',
    'The Ordinary Lactic Acid 10% + HA 2% Volume': 'Exfoliator',
    'The Ordinary- Niacinamide 10% + Zinc 1%': 'Serum',
    'The Ordinary Lactic Acid 5% + HA': 'Exfoliator',
    'AHA 30% + BHA 2% Exfoliating Peeling Solution': 'Mask',
    'Hyaluronic Acid 2% + B5 Hydrating Serum with Ceramides': 'Serum',
    'Natural Moisturizing Factors + Hyaluronic Acid Daily Moisturizer': 'Moisturizer',
    'Skin Care Set': 'Set',
    'Acne Skin Care Set': 'Set',
    'Multi-Peptide Serum for Hair Density': 'Hair Care',
    'Brightening Cream': 'Moisturizer',
    'Eye Brightening Serum': 'Eye Care',
    'Anti-Aging Peptide Serum': 'Serum',
    'Anti-Wrinkle Serum': 'Serum',
    'Anti-Aging Peptide Serum with Hydration': 'Serum',
    'Squalane Cleanser For Dry Skin': 'Cleanser',
    'Soothing & Barrier Support Serum': 'Serum',
    'Glucoside Foaming Facial Cleanser': 'Cleanser',
    '100% Organic Cold-Pressed Rose Hip Seed Oil for Aging Skin': 'Oil',
    'Matrixyl 10% + HA High-Strength Wrinkle Support Serum': 'Serum',
    'Multi-Peptide Anti-Aging Eye Serum': 'Eye Care',
    'Saccharomyces Ferment 30% Milky Toner': 'Toner',
    'Natural Moisturizing Factors + PhytoCeramides Rich Moisturizer': 'Moisturizer',
    '100% Plant-Derived Squalane Skin and Hair Hydrator Serum': 'Oil',
    'Anti-Aging Retinoid Emulsion': 'Serum',
    'Balancing Serum for Blemish Prone Skin': 'Serum',
    'Salicylic Acid 2% Mask for Blemish Prone Skin': 'Mask',
    'Peptide Skin Care Set': 'Set',
    'Vitamin C Suspension 23% + HA Spheres 2% Serum': 'Serum',
    'Lactic Acid 10% + HA 2% Exfoliating Serum': 'Exfoliator',
    'Natural Moisturizing Factors + Beta Glucan Gel Moisturizer': 'Moisturizer',
    'Ascorbyl Glucoside Solution 12% Serum': 'Serum',
    '100% Cold Pressed Virgin Marula Oil': 'Oil',
    'Glycolipid Cream Facial Cleanser': 'Cleanser',
    'Niacinamide 5% Face and Body Emulsion': 'Emulsion',
    'Salicylic Acid 2% Anhydrous Solution Serum': 'Serum',
    'Marine Hyaluronic Moisture Lock Serum': 'Serum',
    'Granactive Retinoid 5% in Squalane Serum': 'Serum',
    '100% Organic Cold-Pressed Moroccan Argan Oil': 'Oil',
    'Aloe 2% + NAG 2% Solution Serum': 'Serum',
    'Natural Moisturizing Factors + Hyaluronic Acid Scalp Serum': 'Scalp Care',
    'Salicylic Acid 0.5% Body Serum': 'Serum',
    'Pycnogenol 5% High-Potency Antioxidant Serum': 'Serum',
    'Amino Acids + B5 Hydrating Serum': 'Serum',
    'Resveratrol 3% + Ferulic Acid 3% Serum': 'Serum',
    '100% Niacinamide High-Potency Oil-Control Powder': 'Powder',
    'Cosrx Acne Pimple Master Patch': 'Acne Patch',
    'COSRX AHA/BHA Clarifying Treatment Toner': 'Toner',
    'Cosrx One Step Original Clear Pad': 'Exfoliator',
    'Cosrx Clear Fit Master Patch': 'Acne Patch',
    'Cosrx BHA Blackhead Power Liquid': 'Exfoliator',
    'COSRX The Vitamin C 23 serum 20ml': 'Serum',
    'Cosrx Advanced Snail 96 Mucin Power Essence': 'Essence',
    'Cosrx Salicylic Acid Daily Gentle Cleanser': 'Cleanser',
    'COSRX - OIL-FREE ULTRA-MOISTURIZING LOTION 100ml': 'Moisturizer',
    'Cosrx Low pH Good Morning Gel Cleanser': 'Cleanser',
    'COSRX - Aloe Soothing Sun Cream SPF50+/ PA+++ 50ml': 'Sunscreen',
    'Advanced Snail 92 All In One Cream': 'Moisturizer',
    'CeraVe Resurfacing Retinol Serum': 'Serum',
    'CeraVe Daily Moisturizing Lotion': 'Moisturizer',
    'CeraVe Hydrating Facial Cleanser': 'Cleanser',
    'CeraVe Hydrating Cream-to-Foam Cleanser': 'Cleanser',
    'CeraVe Renewing SA Cleanser': 'Cleanser',
    'CeraVe Foaming Facial Cleanser': 'Cleanser',
    'CeraVe Hydrating Cleanser for Normal to Dry Skin USA': 'Cleanser',
    'CeraVe Hydrating Foaming Oil Cleanser': 'Cleanser',
    'CeraVe Skin Renewing Night Cream': 'Moisturizer',
    'CeraVe Moisturizing Cream': 'Moisturizer',
    'CeraVe PM Lotion': 'Moisturizer',
    'CeraVe AM Lotion SPF 30': 'Moisturizer/Sunscreen',
    'CeraVe Skin Renewing Vitamin C Serum': 'Serum',
    'CeraVe Skin Renewing Eye Cream': 'Eye Cream',
    'CeraVe Acne Control Cleanser': 'Cleanser',
    'CeraVe Skin Renewing Retinol Serum': 'Serum',
    'CeraVe Hydrating Hyaluronic Acid Serum': 'Serum',
    'CeraVe SA Cream': 'Moisturizer',
    'Eye Repair Cream for Dark Circles and Puffiness for All Skin Types': 'Eye Cream',
    'CeraVe Acne Foaming Cream Cleanser': 'Cleanser',
    'CeraVe Ultra-Light Moisturizing Lotion': 'Moisturizer',
    'Alcohol-Free No Rinse Hydrating Facial Toner for Sensitive & Dry Skin': 'Toner',
    'CeraVe Makeup Removing Wipes': 'Makeup Remover',
    'CeraVe Mineral Sunscreen SPF 30': 'Sunscreen',
    'CeraVe Healing Ointment': 'Treatment',
    'CeraVe Ultra-Light Gel': 'Moisturizer',
    'CeraVe Acne Foaming Cream Wash': 'Cleanser',
    'CeraVe Makeup Removing Cleansing Balm': 'Makeup Remover',
    'CeraVe SA Lotion': 'Moisturizer',
    'CeraVe Skin Renewing Nightly Exfoliating Treatment': 'Treatment',
    'CeraVe Hydrating Micellar Water': 'Makeup Remover',
    'CeraVe Skin Renewing Gel Oil': 'Treatment',
    'CeraVe Skin Renewing Day Cream': 'Moisturizer',
    'Travel Size Daily Moisturizing Lotion for Balanced to Dry Skin': 'Moisturizer',
    'CeraVe AM Lotion SPF 50': 'Moisturizer/Sunscreen',
    'CeraVe Acne Control Gel': 'Treatment',
    'CeraVe Comforting Eye Makeup Remover': 'Makeup Remover',
    'CeraVe Mineral Sunscreen SPF 50': 'Sunscreen',
    'CeraVe Oil Control Gel-Cream': 'Moisturizer',
    'CeraVe Sheer Sunscreen SPF 30': 'Sunscreen'
    
}
active_ingredient_mapping = {
    'Conatural Hair Conditioner': "Aqua, Propylene Glycol, Myristyl Alcohol, Cetrimonium Chloride, Cetearyl Alcohol, Isopropyl Myristate, Camellia Sinensis Leaf Extract, Citrus Sinensis Fruit Extract, Helichrysum Italicum Extract, Panthenol, Hydrolyzed Moringa Oleifera Seed Extract, Hydrolyzed Keratin, Parfum, Allantoin, Sodium Benzoate, Potassium Sorbate, Citral, Coumarin, Eugenol, Limonene, Linalool",
    'Conatural Tea Tree Essential Oil': "Melaleuca alternifolia oil",
    'Conatural Saffron Complexion Builder': "Aqua (Water), Glycerin, Rubia Cordifolia Root Extract, Curcuma Longa Root, Acrylates\\C10-30 Alkyl Acrylate Crosspolymer, Polygonum Cuspidatum Root Extract, Scutellaria Baicalcnsis Root Extract, Camellia Sinensis Leaf Extract, Glycyrrhiza Glabra Root Extract, Caprylyl Glycol, EDTA, Nelumbo Nucifera (Lotus) Seed, Ioscorea Villosa (Wild Yam) Root Extract, Kigela Africana Fruit Extract, Equisetum Giganteum Extract, Crocus Sativus (Saffron) Flower Extract",
    'Conatural Anti-Aging - Super Activs Skin Serum': "Aqua, Dimethyl Isosorbide, Glycerin, Butylene Glycol, Carbomer, Polysorbate 20, Palmitoyl Tripeptide-1, Palmitoyl, Tetrapeptide-7, Acrylates/Beheneth-25, Methacrylate Copolymer, Methylpropanediol, Phenoxyethanol, Caprylyl Glycol, Triethanolamine",
    'Conatural Anti-Pigmentation - Super Activs Skin Serum': "Humulus Lupulus Extract",
    'Conatural Hyaluronic Acid 2% + B5 - Super Activs Skin Serum': "Aqua, Dimethyl Isosorbide, Acrylates/Beheneth-25 Methacrylate Copolymer, Methylpropanediol, Phenoxyethanol, Caprylyl Glycol, D Panthenol, Sodium Hyaluronate, Triethanolamine",
    'Conatural Niacinamide 10% + Zinc 1% - Super Activs Skin Serum': "Aqua, Niacinamide, Dimethyl Isosorbide, Zinc PCA, Methylpropanediol, Phenoxyethanol, Caprylyl Glycol, Hydroxypropyl Methylcellulose",
    'Conatural Rose Face Wash': "Rosa Damascene Flower Water, Glycerin, Chamomille Recutita, Calendula Officinalis, D-Panthenol, Hydroxypropyl Methylcellulose, Sodium Cocoyl Isethionate, Gylcyrrhiza Glabra Extract, Centella Asiatica Extract, Polygonum Cuspidatum Root Extract, Scutellaria Baicalensis Root Extract, Camellia Sinensis Leaf Extract, Dioscorea Villosa Root Extract, Kigela Africana Fruit Extract, Equisetum Giganteum Extract, Fructan, Rosmarinus Officinalis Leaf Extract",
    'The Moms Co. Natural Vita Rich Face Serum with Vitamins and Hyaluronic Acid': "Hyaluronic Acid",
    'The Moms Co Natural Vita Rich Face Serum (8 ml) (Mini / Small ...)': "Vitamins C, Vitamins B3, Vitamins B5, Vitamins E, Hyaluronic Acid, Soybean Phospholipids",
    'The Moms Co. Natural Baby Lotion with Rice Bran, Apricot, Jojoba Oil, Cocoa and Shea Butter': "Rice Bran, Apricot, Jojoba Oil, Cocoa, Shea Butter",
    'The Body Shop Seaweed Oil-Control Face Wash': "Aqua/Water/Eau, Glycerin, Cocamidopropyl Betaine, Sodium Laureth Sulfate, Sodium Chloride, Sorbitol, Sodium Methyl Cocoyl Taurate, PEG-120 Methyl Glucose Dioleate, Phenoxyethanol, Caprylyl Glycol, Sodium Citrate, Polysorbate 20, Parfum/Fragrance, Butylene Glycol, Trisodium Ethylenediamine Disuccinate, Citric Acid, Linalool, Benzyl Salicylate, Aloe Barbadensis Leaf Juice Powder, Laminaria Saccharina Extract, Zinc Sulfate, Pyridoxine HCL, CI 42090/Blue 1",
    'The Body Shop Body Yogurt - Almond Milk': "Aqua, Glycerin, Alcohol Denat., Dimethicone, Butylene Glycol, Parfum, Phenoxyethanol, Carbomer, PEG-100 Stearate, Glyceryl Stearate, Butyrospermum Parkii Butter, Dimethiconol, Caprylyl Glycol, Sodium Hydroxide, Sodium Hyaluronate, Prunus Amygdalus Dulcis Seed Extract, Xanthan Gum, Citric Acid",
    'The Body Shop Himalayan Charcoal': "Aqua/Water/Eau, Propanediol, Alcohol Denat., Glycerin, Hibiscus Sabdariffa Flower Extract, Xanthan Gum, Benzyl Alcohol, Epilobium Fleischeri Extract, Salicylic Acid, Sodium Hydroxide, Parfum/Fragrance, Charcoal Powder, Limonene, Linalool, Citric Acid, Potassium Sorbate",
    'The Body Shop Vitamin C Liquid Peel': "Aqua/Water/Eau, Propanediol, Alcohol Denat., Glycerin, Hibiscus Sabdariffa Flower Extract, Xanthan Gum, Benzyl Alcohol, Epilobium Fleischeri Extract, Salicylic Acid, Sodium Hydroxide, Parfum/Fragrance, Charcoal Powder, Limonene, Linalool, Citric Acid, Potassium Sorbate",
    'The Body Shop Vitamin C Glow Boosting Moisturiser': "Aqua/Water/Eau, Alcohol Denat., Glycerin, Propanediol, Ammonium Acryloyldimethyltaurate/VP Copolymer, Ethyl Macadamiate, Isostearyl Lactate, Phenoxyethanol, Isopropyl Alcohol, Parfum/Fragrance, Sesamum Indicum Seed Oil/Sesamum Indicum (Sesame) Seed Oil, Limonene, Sodium Hyaluronate, Tetrahexyldecyl Ascorbate, Disodium EDTA, Benzyl Salicylate, Hexyl Cinnamal, Linalool, Aloe Barbadensis Leaf Juice Powder, Myrciaria Dubia Fruit Extract, Sodium Hydroxide, Citral, Tocopherol, Malic Acid, CI 15510/Orange 4, CI 15985/Yellow 6",
    'The Body Shop Drops of Youth™ Concentrate': "Aqua/Water/Eau, Glycerin, Propanediol, Alcohol Denat., Fagus Sylvatica Bud Extract, Sodium Hyaluronate, Benzyl Alcohol, Xanthan Gum, Salicylic Acid, Sodium Hydroxide, Adenosine, Parfum/Fragrance, Leontopodium Alpinum Callus Culture Extract, Linalool, Crithmum Maritimum Extract, Eryngium Maritimum Extract, Orbignya Oleifera Seed Oil, Citronellol, Malva Sylvestris Flower Extract/Mallow Flower Extract, Benzoic Acid, Sorbic Acid, Citric Acid, Limonene, Sodium Benzoate, Potassium Sorbate",
    'The Body Shop Vitamin E Gentle Facial Wash': "Aqua/Water (Solvent/Diluent), Glycine Soja Oil/Glycine Soja (Soybean) Oil (Emollient/Skin Conditioner), Cocamidopropyl Betaine (Surfactant), Coco-Glucoside (Surfactant), Triticum Vulgare Germ Oil/Triticum Vulgare (Wheat) Germ Oil (Emollient), Sodium Laureth Sulfate (Surfactant), Acrylates/C10-30 Alkyl Acrylate Crosspolymer (Stabiliser/Viscosity Modifier), Acrylates/Steareth-20 Methacrylate Copolymer (Viscosity Modifier), Phenoxyethanol (Preservative), Tocopheryl Acetate (Antioxidant), Sodium Hydroxide (pH Adjuster), Sodium Benzoate (Preservative), Parfum/Fragrance (Fragrance), Benzyl Benzoate (Fragrance Ingredient), Hydroxycitronellal (Fragrance Ingredient), Disodium EDTA (Chelating Agent), Limonene (Fragrance Ingredient), Geraniol (Fragrance Ingredient), Tocopherol (Antioxidant), Linalool (Fragrance Ingredient), Citronellol (Fragrance Ingredient), Cinnamyl Alcohol (Fragrance Ingredient), CI 17200/Red 33 (Colour), CI 14700/Red 4 (Colorant)",
    'The Body Shop Oils of Life™ Sleeping Cream': "Aqua, Glycerin, Pentaerythritol Pentaerythrityl Tetraethylhexanoate, Dimethicone, Alcohol Denat., Ammonium Acryloyldimethyltaurate/VP Copolymer, Fagus Sylvatica Bud Extract, Phenoxyethanol, Caprylyl Glycol, Dimethiconol, Mannose, Xanthan Gum, Sodium Hyaluronate, Sodium Hydroxide, Parfum, Rosa Canina Seed Oil, Sambucus Nigra Flower Extract, Carthamus Tinctorius (Safflower) Seed Oil, Squalane, Hippophae Rhamnoides Oil, Prunus Armeniaca (Apricot) Kernel Oil, Camellia Sinensis Leaf Extract, Rheum Rhaponticum Root Extract, Lavandula angustifolia (Lavender) Oil, Cinnamomum Zeylanicum Bark Oil, Linalool, Benzyl Alcohol, Citronellol, Citral, Limonene, Hexyl Cinnamal, Eucalyptus Globulus Leaf Oil",
    'The Body Shop Japanese Matcha Tea Pollution Clearing Mask': "Aqua, Charcoal Powder, Glycerin, Cetearyl Alcohol, Ceteareth-20, Ethylhexylglycerin, Sodium Chloride, Kaolin, Lactic Acid, Camellia Sinensis Leaf Powder, Bentonite, Zinc Oxide, Sodium Hyaluronate, Salicylic Acid, Citric Acid, Parfum/Fragrance, Limonene, Linalool, Eugenol, Tocopherol",
    'The Body Shop Drops of Youth™ Concentrate': "Water (Aqua), Glycerin, Propanediol, Alcohol Denat., Fagus Sylvatica Bud Extract, Sodium Hyaluronate, Benzyl Alcohol, Xanthan Gum, Salicylic Acid, Sodium Hydroxide, Adenosine, Parfum/Fragrance, Leontopodium Alpinum Callus Culture Extract, Linalool, Crithmum Maritimum Extract, Eryngium Maritimum Extract, Orbignya Oleifera Seed Oil, Citronellol, Malva Sylvestris Flower Extract/Mallow Flower Extract, Benzoic Acid, Sorbic Acid, Citric Acid, Limonene, Sodium Benzoate, Potassium Sorbate",
    'The Body Shop Vitamin C Liquid Peel': "Water, Glycerin, Propanediol, Alcohol Denat., Glycerin, Sodium Hydroxide, Charcoal Powder, Parfum/Fragrance",
    'The Body Shop Vitamin C Glow Boosting Moisturiser': "Aqua, Glycerin, Alcohol Denat., Dimethicone, Butylene Glycol, Parfum, Phenoxyethanol, Carbomer, Glyceryl Stearate, Sodium Hydroxide, Dimethiconol, Ethylhexylglycerin, Sodium Hyaluronate",
    'The Body Shop Himalayan Charcoal Purifying Glow Mask': "Aqua, Charcoal Powder, Glycerin, Kaolin, Bentonite, Propylene Glycol, Sodium Chloride, Polysorbate 20, Sodium Benzoate, Limonene, Linalool, Geraniol, Citral, Tetrasodium EDTA, Parfum, Green Tea Leaf Extract",
    'The Body Shop Eyeliner Liquid - 01 Black': "Pigment", 
    'Coconut Sugar Body Scrub': "Exfoliant", 
    'Neutrogena Hydro Boost Cleanser Water Gel': "Hyaluronic Acid", 
    'Neutrogena Oil Free Acne Wash': "Salicylic Acid", 'Neutrogena Daily Lip Balm SPF 15': "SPF", 
    'Neutrogena Spot Controlling Facial Wash 200Ml': "Salicylic Acid", 
    'Neutrogena Hydra Boost Hyaluronic Acid Serum': "Hyaluronic Acid", 
    'Neutrogena Ultra Sheer Dry-Touch Sunblock SPF 50+ PA++++': "SPF", 
    'Neutrogena Facial Wash Oil Balancing With Lime & Aloe Vera Pump 200Ml': "Salicylic Acid", 
    'Neutrogena Facial Wash Fresh & Clear With Pink Grapefruit Pump 200Ml': "Salicylic Acid", 
    'Neutrogena Ultra Sheer® Dry-Touch Sunscreen Broad Spectrum SPF 45 (Expiry-3-2024)': "SPF", 
    'Hydro Boost Hyaluronic Acid Water Gel Moisturizer': "Hyaluronic Acid", 
    'Hydro Boost Hydrating 100% Hydrogel Mask': "Hyaluronic Acid", 'The Ordinary Niacinamide 10% + Zinc 1% Serum': "Niacinamide, Zinc", 'The Ordinary Salicylic Acid 2% Solution': "Salicylic Acid", 'The Ordinary Retinol 1% In Squalane': "Retinol", 'The Ordinary Glycolic Acid 7% Toning Solution': "Glycolic Acid", 'The Ordinary Retinol 0.5% In Squalane': "Retinol", 'The Ordinary Vitamin C Suspension 23% + HA Spheres 2%': "Vitamin C, Hyaluronic Acid", 'The Ordinary Retinol 0.2% In Squalane': "Retinol", 'The Ordinary Hyaluronic Acid 2% + B5': "Hyaluronic Acid", 'The Ordinary Lactic Acid 10% + HA 2% Volume': "Lactic Acid, Hyaluronic Acid", 'The Ordinary- Niacinamide 10% + Zinc 1%': "Niacinamide, Zinc", 'The Ordinary Lactic Acid 5% + HA': "Lactic Acid, Hyaluronic Acid", 'AHA 30% + BHA 2% Exfoliating Peeling Solution': "AHA, BHA", 'Hyaluronic Acid 2% + B5 Hydrating Serum with Ceramides': "Hyaluronic Acid, Ceramides", 'Natural Moisturizing Factors + Hyaluronic Acid Daily Moisturizer': "Hyaluronic Acid", 'Skin Care Set': "Multiple Ingredients", 'Acne Skin Care Set': "Salicylic Acid, Niacinamide", 'Multi-Peptide Serum for Hair Density': "Peptides", 'Brightening Cream': "Vitamin C", 'Eye Brightening Serum': "Vitamin C", 'Anti-Aging Peptide Serum': "Peptides", 'Anti-Wrinkle Serum': "Peptides", 'Anti-Aging Peptide Serum with Hydration': "Peptides, Hyaluronic Acid", 'Squalane Cleanser For Dry Skin': "Squalane", 'Soothing & Barrier Support Serum': "Ceramides", 'Glucoside Foaming Facial Cleanser': "Cleansing Agent", '100% Organic Cold-Pressed Rose Hip Seed Oil for Aging Skin': "Rosehip Oil", 'Matrixyl 10% + HA High-Strength Wrinkle Support Serum': "Matrixyl, Hyaluronic Acid", 'Multi-Peptide Anti-Aging Eye Serum': "Peptides", 'Saccharomyces Ferment 30% Milky Toner': "Fermented Ingredients", 'Natural Moisturizing Factors + PhytoCeramides Rich Moisturizer': "Ceramides", '100% Plant-Derived Squalane Skin and Hair Hydrator Serum': "Squalane", 'Anti-Aging Retinoid Emulsion': "Retinoid", 'Balancing Serum for Blemish Prone Skin': "Niacinamide", 'Salicylic Acid 2% Mask for Blemish Prone Skin': "Salicylic Acid", 'Peptide Skin Care Set': "Peptides", 'Vitamin C Suspension 23% + HA Spheres 2% Serum': "Vitamin C, Hyaluronic Acid", 'Lactic Acid 10% + HA 2% Exfoliating Serum': "Lactic Acid, Hyaluronic Acid", 'Natural Moisturizing Factors + Beta Glucan Gel Moisturizer': "Beta Glucan", 'Ascorbyl Glucoside Solution 12% Serum': "Ascorbyl Glucoside", '100% Cold Pressed Virgin Marula Oil': "Marula Oil", 'Glycolipid Cream Facial Cleanser': "Glycolipid", 'Niacinamide 5% Face and Body Emulsion': "Niacinamide", 'Salicylic Acid 2% Anhydrous Solution Serum': "Salicylic Acid", 'Marine Hyaluronic Moisture Lock Serum': "Hyaluronic Acid", 'Granactive Retinoid 5% in Squalane Serum': "Retinoid, Squalane", '100% Organic Cold-Pressed Moroccan Argan Oil': "Argan Oil", 'Aloe 2% + NAG 2% Solution Serum': "Aloe, NAG", 'Natural Moisturizing Factors + Hyaluronic Acid Scalp Serum': "Hyaluronic Acid", 'Salicylic Acid 0.5% Body Serum': "Salicylic Acid", 'Pycnogenol 5% High-Potency Antioxidant Serum': "Pycnogenol", 'Amino Acids + B5 Hydrating Serum': "Amino Acids, B5", 'Resveratrol 3% + Ferulic Acid 3% Serum': "Resveratrol, Ferulic Acid", '100% Niacinamide High-Potency Oil-Control Powder': "Niacinamide", 'Cosrx Acne Pimple Master Patch': "Salicylic Acid", 'COSRX AHA/BHA Clarifying Treatment Toner': "AHA, BHA", 'Cosrx One Step Original Clear Pad': "Salicylic Acid", 'Cosrx Clear Fit Master Patch': "Salicylic Acid", 'Cosrx BHA Blackhead Power Liquid': "BHA", 'COSRX The Vitamin C 23 serum 20ml': "Vitamin C", 'Cosrx Advanced Snail 96 Mucin Power Essence': "Snail Mucin", 'Cosrx Salicylic Acid Daily Gentle Cleanser': "Salicylic Acid", 'COSRX - OIL-FREE ULTRA-MOISTURIZING LOTION 100ml': "Moisturizing Agents", 'Cosrx Low pH Good Morning Gel Cleanser': "Salicylic Acid", 'COSRX - Aloe Soothing Sun Cream SPF50+/ PA+++ 50ml': "Aloe, SPF", 'Advanced Snail 92 All In One Cream': "Snail Mucin", 'CeraVe Resurfacing Retinol Serum': "Retinol", 'CeraVe Daily Moisturizing Lotion': "Ceramides", 'CeraVe Hydrating Facial Cleanser': "Hyaluronic Acid", 'CeraVe Hydrating Cream-to-Foam Cleanser': "Hyaluronic Acid", 'CeraVe Renewing SA Cleanser': "Salicylic Acid", 'CeraVe Foaming Facial Cleanser': "Foaming Agents", 'CeraVe Hydrating Cleanser for Normal to Dry Skin USA': "Hyaluronic Acid", 'CeraVe Hydrating Foaming Oil Cleanser': "Hyaluronic Acid", 'CeraVe Skin Renewing Night Cream': "Ceramides", 'CeraVe Moisturizing Cream': "Ceramides", 'CeraVe PM Lotion': "Ceramides", 'CeraVe AM Lotion SPF 30': "SPF, Ceramides", 'CeraVe Skin Renewing Vitamin C Serum': "Vitamin C", 'CeraVe Skin Renewing Eye Cream': "Ceramides", 'CeraVe Acne Control Cleanser': "Salicylic Acid", 'CeraVe Skin Renewing Retinol Serum': "Retinol", 'CeraVe Hydrating Hyaluronic Acid Serum': "Hyaluronic Acid", 'CeraVe SA Cream': "Salicylic Acid", 'Eye Repair Cream for Dark Circles and Puffiness for All Skin Types': "Caffeine", 'CeraVe Acne Foaming Cream Cleanser': "Salicylic Acid", 'CeraVe Ultra-Light Moisturizing Lotion': "Ceramides", 'Alcohol-Free No Rinse Hydrating Facial Toner for Sensitive & Dry Skin': "Hyaluronic Acid", 'CeraVe Makeup Removing Wipes': "Gentle Cleanser", 'CeraVe Mineral Sunscreen SPF 30': "SPF", 'CeraVe Healing Ointment': "Petrolatum", 'CeraVe Ultra-Light Gel': "Ceramides", 'CeraVe Acne Foaming Cream Wash': "Salicylic Acid", 'CeraVe Makeup Removing Cleansing Balm': "Cleansing Agents", 'CeraVe SA Lotion': "Salicylic Acid", 'CeraVe Skin Renewing Nightly Exfoliating Treatment': "Retinol", 'CeraVe Hydrating Micellar Water': "Micellar Water", 'CeraVe Skin Renewing Gel Oil': "Ceramides", 'CeraVe Skin Renewing Day Cream': "Ceramides", 'Travel Size Daily Moisturizing Lotion for Balanced to Dry Skin': "Ceramides", 'CeraVe AM Lotion SPF 50': "SPF, Ceramides", 'CeraVe Acne Control Gel': "Salicylic Acid", 'CeraVe Comforting Eye Makeup Remover': "Gentle Cleanser", 'CeraVe Mineral Sunscreen SPF 50': "SPF", 'CeraVe Oil Control Gel-Cream': "Niacinamide"
    
}
benefits_mapping = {
    'Conatural Hair Conditioner': 'Strengthens and nourishes hair, adds shine, and improves hair texture.',
    'Conatural Tea Tree Essential Oil': 'Helps control acne, reduces inflammation, and promotes clearer skin.',
    'Conatural Saffron Complexion Builder': 'Brightens skin tone, reduces hyperpigmentation, and enhances radiance.',
    'Conatural Anti-Aging - Super Activs Skin Serum': 'Reduces wrinkles, improves skin elasticity, and boosts hydration.',
    'Conatural Anti-Pigmentation - Super Activs Skin Serum': 'Fades dark spots, lightens pigmentation, and evens skin tone.',
    'Conatural Hyaluronic Acid 2% + B5 - Super Activs Skin Serum': 'Deeply hydrates, plumps the skin, and restores moisture balance.',
    'Conatural Niacinamide 10% + Zinc 1% - Super Activs Skin Serum': 'Minimizes pores, regulates sebum, and improves skin texture.',
    'Conatural Rose Face Wash': 'Gently cleanses, refreshes skin, and leaves a smooth finish.',
        
    "The Ordinary Niacinamide 10% + Zinc 1% Serum": "Reduces Blemishes, Decreases Inflammation & Redness, Hydrates, Fast Absorbing, All Skin Types, AM & PM Use",
    "The Ordinary Vitamin C Suspension 23% + HA Spheres 2% Serum": "Brightens Skin Tone, Reduces Fine Lines, Hydrates, Antioxidant Protection, Suitable for All Skin Types",
    "The Ordinary Alpha Arbutin 2% + HA Serum": "Fades Dark Spots, Evens Skin Tone, Hydrates, Antioxidant Properties, Suitable for All Skin Types",
    "The Ordinary Lactic Acid 10% + HA 2% Serum": "Exfoliates Skin, Improves Skin Texture, Hydrates, Reduces Fine Lines, Suitable for All Skin Types",
    "The Ordinary Retinol 0.2% in Squalane Serum": "Encourages Cell Turnover, Reduces Fine Lines, Improves Skin Texture, Hydrates, Suitable for All Skin Types",
    "The Ordinary Glycolic Acid 7% Toning Solution": "Exfoliates Skin, Improves Skin Texture, Reduces Dullness, Hydrates, Suitable for All Skin Types",
    "The Ordinary Vitamin C Suspension 30% in Silicone Serum": "Brightens Skin Tone, Reduces Fine Lines, Antioxidant Protection, Suitable for All Skin Types",
    "The Ordinary Retinol 0.5% in Squalane Serum": "Encourages Cell Turnover, Reduces Fine Lines, Improves Skin Texture, Hydrates, Suitable for All Skin Types",
    "The Ordinary Retinol 1% in Squalane Serum": "Encourages Cell Turnover, Reduces Fine Lines, Improves Skin Texture, Hydrates, Suitable for All Skin Types",
    'The Ordinary Salicylic Acid 2% Solution': 'Exfoliator',
    'The Ordinary Retinol 1% In Squalane': "Encourages Cell Turnover, Reduces Fine Lines, Improves Skin Texture, Hydrates, Suitable for All Skin Types",
    'The Ordinary Retinol 0.2% In Squalane': "Encourages Cell Turnover, Reduces Fine Lines, Improves Skin Texture, Hydrates, Suitable for All Skin Types",
    'The Ordinary Hyaluronic Acid 2% + B5': 'provides instant plumping hydration to give softer, smoother, healthy-looking skin',
    'The Ordinary Lactic Acid 10% + HA 2% Volume': "Exfoliates Skin, Improves Skin Texture, Hydrates, Reduces Fine Lines, Suitable for All Skin Types",
    'The Ordinary- Niacinamide 10% + Zinc 1%': "Reduces Blemishes, Decreases Inflammation & Redness, Hydrates, Fast Absorbing, All Skin Types, AM & PM Use",
    'The Ordinary Lactic Acid 5% + HA': 'exfoliate the skin and promote a brighter, smoother complexion',
    'AHA 30% + BHA 2% Exfoliating Peeling Solution': 'improve skin texture, reduce pore congestion, and address uneven skin tone',
    'Hyaluronic Acid 2% + B5 Hydrating Serum with Ceramides': 'hydrate, smooth and visibly reduce lines and wrinkles',
    'Natural Moisturizing Factors + Hyaluronic Acid Daily Moisturizer': 'promotes smoother, softer skin with improved moisture retention',
    'Skin Care Set': 'mproves the look of skin clarity, texture, radiance, and suppleness.',
    'Acne Skin Care Set': 'prevent acne',
    'Multi-Peptide Serum for Hair Density': 'Hair Care',
    'Brightening Cream': 'targets dullness, uneven texture, and acne blemishes',
    'Eye Brightening Serum': 'improve the look of tired eyes while providing a brightening effect',
    'Anti-Aging Peptide Serum': 'Targets common signs of aging such as the look of crow\'s feet, firmness and elasticity',
    'Anti-Wrinkle Serum': 'Targets common signs of aging such as the look of crow\'s feet, firmness and elasticity',
    'Anti-Aging Peptide Serum with Hydration': 'Targets common signs of aging such as the look of crow\'s feet, firmness and elasticity',
    'Squalane Cleanser For Dry Skin': 'hydrated, smooth skin ',
    'Soothing & Barrier Support Serum': 'repair the skin barrier, soothe skin discomfort, and reduce the appearance of redness',
    'Glucoside Foaming Facial Cleanser': 'remove dirt and environmental impurities',
    '100% Organic Cold-Pressed Rose Hip Seed Oil for Aging Skin': 'naturally hydrates skin to help address signs of aging by brightening and balancing uneven skin tone, supporting skin elasticity, and promoting supple, more radiant-looking skin',
    'Matrixyl 10% + HA High-Strength Wrinkle Support Serum': 'targets fine lines and wrinkles by promoting firmer, plumper skin',
    'Multi-Peptide Anti-Aging Eye Serum': 'educe the appearance of crow\'s feet and under eye wrinkles, eye bags, puffiness, and dark circles, leaving the eye area looking brighter, smoother and less puffy',
    'Saccharomyces Ferment 30% Milky Toner': 'Gently exfoliates for smoother, more hydrated and radiant skin.',
    'Natural Moisturizing Factors + PhytoCeramides Rich Moisturizer': 'boosts skin hydration by 68%',
    '100% Plant-Derived Squalane Skin and Hair Hydrator Serum': 'Hydrates hair',
    'Anti-Aging Retinoid Emulsion': 'combat signs of aging, improve skin texture, reduce the appearance of fine lines and wrinkles, and promote clearer skin',
    'Balancing Serum for Blemish Prone Skin': 'minimizes excess shine, and helps reduce the look of enlarged pores',
    'Salicylic Acid 2% Mask for Blemish Prone Skin': 'removes dead skin cells and excess oil, which promotes soft and smooth skin',
    'Peptide Skin Care Set': 'Targets common signs of aging such as the look of crow\'s feet, firmness and elasticity',
    'Vitamin C Suspension 23% + HA Spheres 2% Serum': 'help visibly reduce signs of aging by brightening and balancing uneven skin tone',
    'Lactic Acid 10% + HA 2% Exfoliating Serum': 'reduces pigmentation, minimises fine lines and wrinkles and improves overall skin texture',
    'Natural Moisturizing Factors + Beta Glucan Gel Moisturizer': ' strengthen the skin barrier, hydrates all day',
    'Ascorbyl Glucoside Solution 12% Serum': 'protect against free radical damage,repair the effects of UV exposure,brighten skin by reducing melanin production,increase collagen levels to improve skin elasticity.',
    '100% Cold Pressed Virgin Marula Oil': ' restore radiant tone',
    'Glycolipid Cream Facial Cleanser': 'Cleanse, remove impurities',
    'Niacinamide 5% Face and Body Emulsion': 'Evens skin tone, reduce drk spots',
    'Salicylic Acid 2% Anhydrous Solution Serum': 'clear up pores, improve the appearance of visible redness and comedones',
    'Marine Hyaluronic Moisture Lock Serum': 'hydrates, soften, smooth skin',
    'Granactive Retinoid 5% in Squalane Serum': 'fine lines, wrinkles, uneven skin tone, and visible pores',
    '100% Organic Cold-Pressed Moroccan Argan Oil': 'restore radiant tone',
    'Aloe 2% + NAG 2% Solution Serum': 'diminishing the look of post-acne marks, uneven skin tone, and texture, while also aiding in minimising the appearance of pores and fortifying the skin barrier',
    'Natural Moisturizing Factors + Hyaluronic Acid Scalp Serum': 'Scalp Care',
    'Salicylic Acid 0.5% Body Serum': 'treat acne and allow skin to heal',
    'Pycnogenol 5% High-Potency Antioxidant Serum': 'helps support your skin against the impact of oxidative stress',
    'Amino Acids + B5 Hydrating Serum': 'Surface and below surface hydration',
    'Resveratrol 3% + Ferulic Acid 3% Serum': 'Serum',
    '100% Niacinamide High-Potency Oil-Control Powder': 'brighter, more radiant complexion,smooth textural irregularities, mattify the skin and minimizing oils.',
    "CeraVe Resurfacing Retinol Serum": "Reduces Dark Spots, Minimizes Fine Lines, Smooths Skin Texture, Hydrates, Suitable for All Skin Types",
    "CeraVe Daily Moisturizing Lotion": "Hydrates, Restores Skin Barrier, Non-Greasy, Lightweight, Suitable for All Skin Types",
    "CeraVe Hydrating Facial Cleanser": "Hydrates, Removes Dirt and Oil, Non-Foaming, Suitable for Dry to Normal Skin",
    "CeraVe Hydrating Cream-to-Foam Cleanser": "Removes Dirt and Oil, Hydrates, Foaming Formula, Suitable for Normal to Oily Skin",
    "CeraVe Renewing SA Cleanser": "Exfoliates Skin, Removes Dead Skin Cells, Hydrates, Helps Prevent Acne, Suitable for Oily Skin",
    "CeraVe Foaming Facial Cleanser": "Removes Dirt and Oil, Non-Drying, Hydrates, Suitable for Normal to Oily Skin",
    "CeraVe Hydrating Cleanser for Normal to Dry Skin USA": "Hydrates, Soothes Skin, Non-Foaming, Suitable for Normal to Dry Skin",
    "CeraVe Hydrating Foaming Oil Cleanser": "Cleanses, Hydrates, Non-Drying, Suitable for Dry Skin",
    "CeraVe Skin Renewing Night Cream": "Moisturizes, Improves Skin Texture, Reduces Fine Lines, Suitable for All Skin Types",
    "CeraVe Moisturizing Cream": "Deeply Hydrates, Restores Skin Barrier, Non-Greasy, Suitable for Dry Skin",
    "CeraVe PM Lotion": "Hydrates, Restores Skin Barrier, Lightweight, Suitable for Nighttime Use",
    "CeraVe AM Lotion SPF 30": "Hydrates, Protects from UV Rays, Restores Skin Barrier, Suitable for Daily Use",
    "CeraVe Skin Renewing Vitamin C Serum": "Brightens Skin, Reduces Dark Spots, Antioxidant Protection, Hydrates",
    "CeraVe Skin Renewing Eye Cream": "Reduces Puffiness, Hydrates, Smooths Fine Lines, Suitable for Sensitive Skin",
    "CeraVe Acne Control Cleanser": "Reduces Acne, Removes Dirt and Oil, Non-Drying, Suitable for Acne-Prone Skin",
    "CeraVe Skin Renewing Retinol Serum": "Encourages Cell Turnover, Reduces Fine Lines, Improves Skin Texture, Hydrates",
    "CeraVe Hydrating Hyaluronic Acid Serum": "Hydrates, Plumps Skin, Reduces Fine Lines, Suitable for All Skin Types",
    "CeraVe SA Cream": "Exfoliates Skin, Softens Skin, Hydrates, Suitable for Rough & Bumpy Skin",
    "Eye Repair Cream for Dark Circles and Puffiness for All Skin Types": "Reduces Dark Circles, Smooths Fine Lines, Hydrates, Suitable for All Skin Types",
    "CeraVe Acne Foaming Cream Cleanser": "Reduces Acne, Removes Dirt and Oil, Non-Drying, Suitable for Acne-Prone Skin",
    "CeraVe Ultra-Light Moisturizing Lotion": "Hydrates, Lightweight, Non-Greasy, Suitable for Oily Skin",
    "Alcohol-Free No Rinse Hydrating Facial Toner for Sensitive & Dry Skin": "Hydrates, Soothes Skin, Alcohol-Free, Suitable for Sensitive Skin",
    "CeraVe Makeup Removing Wipes": "Removes Makeup, Hydrates, Non-Irritating, Suitable for All Skin Types",
    "CeraVe Mineral Sunscreen SPF 30": "Broad Spectrum UV Protection, Hydrates, Non-Greasy, Suitable for Sensitive Skin",
    "CeraVe Healing Ointment": "Protects Dry Skin, Locks in Moisture, Restores Skin Barrier, Suitable for Sensitive Skin",
    "CeraVe Ultra-Light Gel": "Hydrates, Lightweight, Non-Greasy, Suitable for Oily Skin",
    "CeraVe Acne Foaming Cream Wash": "Reduces Acne, Removes Dirt and Oil, Non-Drying, Suitable for Acne-Prone Skin",
    "CeraVe Makeup Removing Cleansing Balm": "Removes Makeup, Hydrates, Gentle, Suitable for All Skin Types",
    "CeraVe SA Lotion": "Exfoliates Skin, Softens Skin, Hydrates, Suitable for Rough & Bumpy Skin",
    "CeraVe Skin Renewing Nightly Exfoliating Treatment": "Exfoliates Skin, Improves Skin Texture, Hydrates, Suitable for All Skin Types",
    "CeraVe Hydrating Micellar Water": "Removes Makeup, Hydrates, Non-Irritating, Suitable for Sensitive Skin",
    "CeraVe Skin Renewing Gel Oil": "Hydrates, Locks in Moisture, Non-Greasy, Suitable for Oily Skin",
    "CeraVe Skin Renewing Day Cream": "Moisturizes, Smooths Fine Lines, Protects Skin, Suitable for All Skin Types",
    "Travel Size Daily Moisturizing Lotion for Balanced to Dry Skin": "Hydrates, Restores Skin Barrier, Non-Greasy, Suitable for All Skin Types",
    "CeraVe AM Lotion SPF 50": "Hydrates, Protects from UV Rays, Restores Skin Barrier, Suitable for Daily Use",
    "CeraVe Acne Control Gel": "Reduces Acne, Targets Blemishes, Non-Drying, Suitable for Acne-Prone Skin",
    "CeraVe Comforting Eye Makeup Remover": "Removes Makeup, Hydrates, Non-Irritating, Suitable for Sensitive Skin",
    "CeraVe Mineral Sunscreen SPF 50": "Broad Spectrum UV Protection, Hydrates, Non-Greasy, Suitable for Sensitive Skin",
    "CeraVe Oil Control Gel-Cream": "Controls Oil, Hydrates, Non-Greasy, Suitable for Oily Skin",
    "CeraVe Sheer Sunscreen SPF 30": "Broad Spectrum UV Protection, Hydrates, Lightweight, Suitable for All Skin Types",
    'The Body Shop Seaweed Oil-Control Face Wash': 'Removes excess oil, refines pores, refreshes skin, suitable for oily skin',
    'The Body Shop Body Yogurt - Almond Milk': 'Hydrates, softens skin, lightweight, non-greasy, suitable for dry skin',
    'The Body Shop Himalayan Charcoal': 'Deep cleanses, detoxifies, minimizes pores, refines skin texture',
    'The Body Shop Vitamin C Liquid Peel': 'Exfoliates, removes dead skin cells, brightens, smooths skin texture',
    'The Body Shop Vitamin C Glow Boosting Moisturiser': 'Brightens, hydrates, gives radiance, suitable for dull skin',
    'The Body Shop Drops of Youth™ Concentrate': 'Reduces signs of aging, hydrates, firms skin, suitable for all skin types',
    'The Body Shop Vitamin E Gentle Facial Wash': 'Cleanses, hydrates, soothes skin, suitable for sensitive skin',
    'The Body Shop Oils of Life™ Sleeping Cream': 'Repairs, rejuvenates, restores skin overnight, suitable for all skin types',
    'The Body Shop Tea Tree Cleansing Facial Wash': 'Cleanses, removes impurities, controls oil, suitable for acne-prone skin',
    'The Body Shop Tea Tree Squeaky Clean Scrub': 'Exfoliates, removes blemishes, controls oil, suitable for acne-prone skin',
    'The Body Shop Vitamin E Overnight Serum-In-Oil': 'Repairs, hydrates, smooths fine lines, suitable for dry skin',
    'The Body Shop Tea Tree Anti-Imperfection Daily Solution': 'Fights imperfections, hydrates, controls oil, suitable for acne-prone skin',
    'The Body Shop Vitamin C Daily Glow Cleansing Polish': 'Exfoliates, brightens, smooths skin texture, suitable for dull skin',
    'The Body Shop Tea Tree Skin Clearing Night Mask': 'Treats blemishes, hydrates, controls oil, suitable for acne-prone skin',
    'The Body Shop Chinese Ginseng and Rice Clarifying Polishing Mask': 'Brightens, removes dead skin cells, suitable for all skin types',
    'The Body Shop Drops Of Light™ Brightening Day Cream': 'Brightens, evens skin tone, hydrates, suitable for dull skin',
    'The Body Shop Tea Tree Skin Clearing Mattifying Toner': 'Reduces oil, minimizes pores, fights blemishes, suitable for oily skin',
    'The Body Shop Skin Defence Multi-Protection Lotion SPF 50+ PA++++': 'Protects from UV rays, hydrates, non-greasy, suitable for sensitive skin',
    'The Body Shop Drops of Light™ Brightening Pure Clarifying Foam Wash': 'Cleanses, brightens, removes impurities, suitable for dull skin',
    'The Crème Shop x Hello Kitty Fusion Body Scrub - Caramel Pumpkin Latte': 'Exfoliates, smooths skin, moisturizes, refreshing scent',
    'The Body Shop Eyeliner Liquid - 01 Black': 'Intense color, long-lasting, smudge-proof, suitable for all skin types',
    'Coconut Sugar Body Scrub': 'Exfoliates, smooths skin, nourishes, suitable for all skin types',

    'COSRX Acne Pimple Master Patch': 'Protects acne, speeds up healing process.',
    'COSRX AHA/BHA Clarifying Treatment Toner': 'Exfoliate surface of skin, Smooths and softens, Clear congested pores, Balance oil production',
    'Cosrx One Step Original Clear Pad': 'treat problematic or acne-prone skin',
    'Cosrx Clear Fit Master Patch': 'reduces inflammation',
    'Cosrx BHA Blackhead Power Liquid': 'Reduces blackheads permanently, Clears the pores & removes dead skin cells, For an even-looking complexion',
    'COSRX The Vitamin C 23 serum 20ml': '- Hyperpigmentation, Anti aging, Dullness, Uneven skin tone, Dark spots',
    'Cosrx Advanced Snail 96 Mucin Power Essence': 'Fades dark spots,Improves skin texture, Anti-aging, Intense hydratio',
    'Cosrx Salicylic Acid Daily Gentle Cleanser': 'acn breakouts, clearer skin',
    'COSRX - OIL-FREE ULTRA-MOISTURIZING LOTION 100ml': 'Soothing irritated skin,Restoring moisture and maintaining hydration level',
    'Cosrx Low pH Good Morning Gel Cleanser': 'Hydrating cleanse without stripping,Removes impurities,Protects skin barrier,Restores pH Balance',
    'COSRX - Aloe Soothing Sun Cream SPF50+/ PA+++ 50ml': 'Sunscreen',
    'Advanced Snail 92 All In One Cream': 'Nourishes & plumps,Anti-aging,Repairs damaged skin,Long lasting hydration',   
    'Neutrogena Ultra Sheer Dry-Touch Sunblock SPF 50+': 'Provides high sun protection, lightweight feel.',
    'Neutrogena Hydro Boost Cleanser Water Gel': 'Hydrates,Moisturize,CLeanse',
    'Neutrogena Oil Free Acne Wash': 'Anti acne',
    'Neutrogena Daily Lip Balm SPF 15': 'Softens, smoothes, protects, conditions dry, chapped lips',
    'Neutrogena Spot Controlling Facial Wash 200Ml': 'clears breakouts',
    'Neutrogena Hydra Boost Hyaluronic Acid Serum': 'Boosts hydration all day',
    'Neutrogena Ultra Sheer Dry-Touch Sunblock SPF 50+ PA++++': 'UVA/UVB Protection, dry touch, oil free, no white cast,non greasy, water resistant',
    'Neutrogena Facial Wash Oil Balancing With Lime & Aloe Vera Pump 200Ml': 'Tightens pores,Oil-free,Provides a mattifying effect',
    'Neutrogena Facial Wash Fresh & Clear With Pink Grapefruit Pump 200Ml': 'Tackles Blemishes, for a clearer, radiant complexion,Does not over dry skin, won\'t clog pores',
    'Neutrogena Ultra Sheer® Dry-Touch Sunscreen Broad Spectrum SPF 45 ( Expiry-3-2024 )': 'UVA/UVB Protection, dry touch, oil free, no white cast,non greasy, water resistant',
    'Hydro Boost Hyaluronic Acid Water Gel Moisturizer': 'instantly delivers deep, weightless hydration for refreshed, quenched, dewy skin',
    'Hydro Boost Hydrating 100% Hydrogel Mask': 'Moisturize, smoothes',
    'The Body Shop Vitamin C Glow Boosting Moisturiser': 'Brightens skin, boosts hydration.',
    
}
df['Skin Type'] = df['Name'].map(skin_type_mapping)
df['Category'] = df['Name'].map(category_mapping)
df['Active Ingredient'] = df['Name'].map(active_ingredient_mapping)
df['Benefits'] = df['Name'].map(benefits_mapping)
print(df.head())

          Brand                                          Name     Price  \
0     Conatural                    Conatural Hair Conditioner       NaN   
1     Conatural              Conatural Tea Tree Essential Oil       NaN   
2     Conatural          Conatural Saffron Complexion Builder       NaN   
3         Cosrx                Cosrx Acne Pimple Master Patch    Rs.849   
4  The Ordinary  The Ordinary Niacinamide 10% + Zinc 1% Serum  Rs.2,799   

                                          ReviewText  Rating  \
0                         Conatural Hair Conditioner     NaN   
1                               Product is very nice     NaN   
2  Very good cream, not irritating, although has ...     NaN   
3        Cosrx Acne Pimple Master Patch (24 Patches)     NaN   
4       The Ordinary Niacinamide 10% + Zinc 1% Serum     NaN   

                   Skin Type       Category  \
0              All/Hair Care      Hair Care   
1            Oily/Acne-prone  Essential Oil   
2  All/Dry/Hyperpigment

In [19]:
df = df.dropna()

In [20]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11992 entries, 258 to 12711
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Brand              11992 non-null  object 
 1   Name               11992 non-null  object 
 2   Price              11992 non-null  object 
 3   ReviewText         11992 non-null  object 
 4   Rating             11992 non-null  float64
 5   Skin Type          11992 non-null  object 
 6   Category           11992 non-null  object 
 7   Active Ingredient  11992 non-null  object 
 8   Benefits           11992 non-null  object 
dtypes: float64(1), object(8)
memory usage: 936.9+ KB


,Brand,Name,Price,ReviewText,Rating,Skin Type,Category,Active Ingredient,Benefits
258,The Ordinary,The Ordinary Hyaluronic Acid 2% + B5,"PKR 3,999.00",It's original and after just use of one week l...,5.0,All,Serum,Hyaluronic Acid,provides instant plumping hydration to give so...
259,The Ordinary,The Ordinary Hyaluronic Acid 2% + B5,"PKR 3,999.00",I have bought original product multiple times ...,1.0,All,Serum,Hyaluronic Acid,provides instant plumping hydration to give so...
260,The Ordinary,The Ordinary Hyaluronic Acid 2% + B5,"PKR 3,999.00",Bohat acha hai best for skin care bigners Rean...,5.0,All,Serum,Hyaluronic Acid,provides instant plumping hydration to give so...
261,The Ordinary,The Ordinary Hyaluronic Acid 2% + B5,"PKR 3,999.00","Excellent result, little amount is enough, goo...",5.0,All,Serum,Hyaluronic Acid,provides instant plumping hydration to give so...
262,The Ordinary,The Ordinary Hyaluronic Acid 2% + B5,"PKR 3,999.00","It really work and orignal product,pleased to ...",5.0,All,Serum,Hyaluronic Acid,provides instant plumping hydration to give so...


In [21]:
# Define the conversion rate for USD to PKR
usd_to_pkr_rate = 280  # You can adjust this value as per the current exchange rate

# Function to extract the numeric price value and currency
def extract_price_and_currency(price):
    price = str(price)  # Ensure it's a string
    # Remove commas and extra text, and get only numeric part
    price_cleaned = ''.join(c for c in price if c.isdigit() or c == '.').strip()

    # Check if the price is in USD or PKR
    if 'USD' in price:
        return float(price_cleaned), 'USD'
    elif 'PKR' in price:
        return float(price_cleaned), 'PKR'
    return None, None

# Apply the extraction function to the 'Price' column
df[['Price_Value', 'currency']] = df['Price'].apply(lambda x: pd.Series(extract_price_and_currency(x)))

# Function to convert USD to PKR
def convert_usd_to_pkr(row):
    if row['currency'] == 'USD':  # If the price is in USD
        return row['Price_Value'] * usd_to_pkr_rate  # Convert to PKR
    return row['Price_Value']  # If the price is already in PKR, keep it

# Apply the conversion function to the price column
df['price_in_pkr'] = df.apply(convert_usd_to_pkr, axis=1)

# Verify the results
print(df[['Price', 'Price_Value', 'currency', 'price_in_pkr']])

              Price  Price_Value currency  price_in_pkr
258    PKR 3,999.00       3999.0      PKR        3999.0
259    PKR 3,999.00       3999.0      PKR        3999.0
260    PKR 3,999.00       3999.0      PKR        3999.0
261    PKR 3,999.00       3999.0      PKR        3999.0
262    PKR 3,999.00       3999.0      PKR        3999.0
...             ...          ...      ...           ...
12707         $4.99          NaN     None           NaN
12708         $4.99          NaN     None           NaN
12709         $4.99          NaN     None           NaN
12710         $4.99          NaN     None           NaN
12711         $4.99          NaN     None           NaN

[11992 rows x 4 columns]


In [22]:
# Split the dataframe into two: products and reviews
products_df = df[['Brand', 'Name', 'price_in_pkr', 'Skin Type', 'Category', 'Active Ingredient', 'Benefits']].drop_duplicates()
reviews_df = df[['Name', 'ReviewText', 'Rating']]
reviews_df['Sentiment'] = None
reviews_df['Confidence_score'] = None
# Saving the data to CSV 
products_df.to_csv('products.csv', index=False)
reviews_df.to_csv('reviews.csv', index=False)

C:\Users\Hobopk\AppData\Local\Temp/ipykernel_16204/2273127408.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['Sentiment'] = None
C:\Users\Hobopk\AppData\Local\Temp/ipykernel_16204/2273127408.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['Confidence_score'] = None


In [ ]:
# Building Translator for reviews in roman urdu
def is_roman_urdu(text):
    # A set of common Roman Urdu words to check for
    roman_urdu_keywords = ["acha", "Acha" "hai", "laga", "kya", "bohot", "acha", "khana", "mujhy", "Mujhy"]
    
    # Check if any of these keywords appear in the text
    return any(keyword in text.lower() for keyword in roman_urdu_keywords)

# Apply the function to detect Roman Urdu reviews
reviews_df['is_roman_urdu'] = reviews_df['ReviewText'].apply(is_roman_urdu)

# Filter reviews that are likely Roman Urdu
roman_urdu_reviews_df = reviews_df[reviews_df['is_roman_urdu'] == True]
roman_urdu_reviews_df.head()


In [ ]:
pip install googletrans==4.0.0-rc1


In [ ]:
pip show googletrans


In [ ]:
pip install translate


In [ ]:
from googletrans import Translator

# Function to translate Roman Urdu to English
def translate_roman_urdu(text):
    translator = Translator()
    return translator.translate(text, src='ur', dest='en').text

# Apply the translation to reviews identified as Roman Urdu
roman_urdu_reviews_df['translated_review'] = roman_urdu_reviews_df['ReviewText'].apply(
    lambda x: translate_roman_urdu(x) if is_roman_urdu(x) else x
)


In [ ]:
roman_urdu_reviews_df.head()
# Create a mapping of Roman Urdu reviews and their translations
translation_mapping = dict(zip(roman_urdu_reviews_df['ReviewText'], roman_urdu_reviews_df['translated_review']))

# Define a function to replace Roman Urdu reviews with translations
def replace_with_translation(review):
    # If the review is in the translation mapping, replace it with the translation
    return translation_mapping.get(review, review)

# Apply the function to the `review_text` column of the original DataFrame
reviews_df['ReviewText'] = reviews_df['ReviewText'].apply(replace_with_translation)

# Verify the results
reviews_df.head()

In [ ]:
# Filter reviews that are likely Roman Urdu
roman_urdu_reviews_df = reviews_df[reviews_df['is_roman_urdu'] == True]
roman_urdu_reviews_df.head()
roman_urdu_reviews_df.shape